# 0465 Hard 465 Optimal Account Balancing

In [ ]:
# Time:  O(n * 2^n), n is the size of the debt.
# Space: O(2^n)

import collections


class Solution(object):
    def minTransfers(self, transactions):
        """
        :type transactions: List[List[int]]
        :rtype: int
        """
        accounts = collections.defaultdict(int)
        for src, dst, amount in transactions:
            accounts[src] += amount
            accounts[dst] -= amount

        debts = [account for account in accounts.itervalues() if account]

        dp = [0]*(2**len(debts))
        sums = [0]*(2**len(debts))
        for i in xrange(len(dp)):
            bit = 1
            for j in xrange(len(debts)):
                if (i & bit) == 0:
                    nxt = i | bit
                    sums[nxt] = sums[i]+debts[j]
                    if sums[nxt] == 0:
                        dp[nxt] = max(dp[nxt], dp[i]+1)
                    else:
                        dp[nxt] = max(dp[nxt], dp[i])
                bit <<= 1
        return len(debts)-dp[-1]

# 0466  Hard  466. Count The Repetitions

In [ ]:
class Solution(object):
    def getMaxRepetitions(self, s1, n1, s2, n2):
        start = {} # s2_idx : s1_round, s2_round
        s1_round, s2_round, s2_idx = 0, 0, 0
        while s1_round < n1:
            s1_round += 1
            for ch in s1:
                if ch == s2[s2_idx]:
                    s2_idx += 1
                    if s2_idx == len(s2):
                        s2_round += 1
                        s2_idx = 0
            if s2_idx in start:
                prev_s1_round, prev_s2_round = start[s2_idx]
                circle_s1_round, circle_s2_round = s1_round - prev_s1_round, s2_round - prev_s2_round
                res = (n1 - prev_s1_round) // circle_s1_round * circle_s2_round
                left_s1_round = (n1 - prev_s1_round) % circle_s1_round + prev_s1_round
                for key, val in start.items():
                    if val[0] == left_s1_round:
                        res += val[1]
                        break
                return res // n2
            else:
                start[s2_idx] = (s1_round, s2_round)
        return s2_round // n2

# 0466 Hard 466 Count The Repetitions

In [ ]:
# Time:  O(s1 * min(s2, n1))
# Space: O(s2)

class Solution(object):
    def getMaxRepetitions(self, s1, n1, s2, n2):
        """
        :type s1: str
        :type n1: int
        :type s2: str
        :type n2: int
        :rtype: int
        """
        repeat_count = [0] * (len(s2)+1)
        lookup = {}
        j, count = 0, 0
        for k in xrange(1, n1+1):
            for i in xrange(len(s1)):
                if s1[i] == s2[j]:
                    j = (j + 1) % len(s2)
                    count += (j == 0)

            if j in lookup:   # cyclic
                i = lookup[j]
                prefix_count = repeat_count[i]
                pattern_count = (count - repeat_count[i]) * ((n1 - i) // (k - i))
                suffix_count = repeat_count[i + (n1 - i) % (k - i)] - repeat_count[i]
                return (prefix_count + pattern_count + suffix_count) / n2
            lookup[j] = k
            repeat_count[k] = count

        return repeat_count[n1] / n2  # not cyclic iff n1 <= s2

# 0471  Hard  471. Encode String with Shortest Length

In [ ]:
class Solution:
    def encode(self, s: str) -> str:
        def dfs(i, j):
            if i == j: return s[i]
            if (i, j) not in memo:
                c1 = min((dfs(i, k) + dfs(k + 1, j) if s[i:k + 1] != s[k + 1:j + 1] else '2[' + dfs(i, k) + ']' for k in range(i, j)), key = len)
                c2 = s[i:j + 1]
                memo[(i, j)] = min(c1, c2, key = len)
                for k in range(i, i + (j - i) // 2 + 1):
                    tar, ind, cnt = s[i:k + 1], i, 0
                    while ind + k - i <= j and s[ind:ind + k - i + 1] == tar:
                        cnt += 1
                        ind += k - i + 1
                    c3 = str(cnt) + '[' + tar + ']' + dfs(ind, j) if ind <= j else str(cnt) + '[' + tar + ']'
                    memo[(i, j)] = min(memo[(i, j)], c3, key = len)
            return memo[(i, j)]
        memo = {}
        return dfs(0, len(s) - 1)

# 0471 Hard 471 Encode String with Shortest Length

In [ ]:
# Time:  O(n^3) on average
# Space: O(n^2)

class Solution(object):
    def encode(self, s):
        """
        :type s: str
        :rtype: str
        """
        def encode_substr(dp, s, i, j):
            temp = s[i:j+1]
            pos = (temp + temp).find(temp, 1)  # O(n) on average
            if pos >= len(temp):
                return temp
            return str(len(temp)/pos) + '[' + dp[i][i + pos - 1] + ']'

        dp = [["" for _ in xrange(len(s))] for _ in xrange(len(s))]
        for length in xrange(1, len(s)+1):
            for i in xrange(len(s)+1-length):
                j = i+length-1
                dp[i][j] = s[i:i+length]
                for k in xrange(i, j):
                    if len(dp[i][k]) + len(dp[k+1][j]) < len(dp[i][j]):
                        dp[i][j] = dp[i][k] + dp[k+1][j]
                encoded_string = encode_substr(dp, s, i, j)
                if len(encoded_string) < len(dp[i][j]):
                    dp[i][j] = encoded_string
        return dp[0][len(s) - 1]

# 0472  Hard  472. Concatenated Words

In [ ]:
class Solution:
    def findAllConcatenatedWordsInADict(self, words):
        def check(w, st):
            if w in st: return True
            for i in range(1, len(w)):
                if w[:i] in st and check(w[i:], st): return True
            return False
        w_set, res = set(words), []
        for w in words:
            w_set.remove(w)
            if check(w, w_set): res += w,
            w_set.add(w)
        return res

# 0472 Hard 472 Concatenated Words

In [ ]:
# Time:  O(n * l^2)
# Space: O(n * l)

class Solution(object):
    def findAllConcatenatedWordsInADict(self, words):
        """
        :type words: List[str]
        :rtype: List[str]
        """
        lookup = set(words)
        result = []
        for word in words:
            dp = [False] * (len(word)+1)
            dp[0] = True
            for i in xrange(len(word)):
                if not dp[i]:
                    continue

                for j in xrange(i+1, len(word)+1):
                    if j - i < len(word) and word[i:j] in lookup:
                        dp[j] = True

                if dp[len(word)]:
                    result.append(word)
                    break

        return result

# 0479  Hard  479. Largest Palindrome Product

In [ ]:
class Solution:
    def largestPalindrome(self, n):
        """
        :type n: int
        :rtype: int
        """
        return list(num for i,num in enumerate([0,9,987,123,597,677,1218,877,475]) if i==n)[0]

# 0479 Hard 479. Largest Palindrome Product

In [ ]:
class Solution(object):
    # def largestPalindrome(self, n):
    #     """
    #     :type n: int
    #     :rtype: int
    #     """
    #     if n == 1:
    #         return 9
    #     # https://leetcode.com/problems/largest-palindrome-product/discuss/96297/Java-Solution-using-assumed-max-palindrom
    #     upperBound = 10 ** n - 1
    #     lowerBound = upperBound / 10
    #     maxNum = upperBound * upperBound
    #     firstHalf = maxNum / (10 ** n)
    #     palindromFound = False
    #     palindrom = 0
    #     while not palindromFound:
    #         palindrom = int(str(firstHalf) + str(firstHalf)[::-1])
    #         for i in xrange(upperBound, lowerBound, -1):
    #             if i * i < palindrom:
    #                 break
    #             if palindrom % i == 0:
    #                 palindromFound = True
    #                 break
    #         firstHalf -= 1
    #     return palindrom % 1337

    def largestPalindrome(self, n):
        # https://leetcode.com/problems/largest-palindrome-product/discuss/96305/Python-Solution-Using-Math-In-48ms
        # https://leetcode.com/problems/largest-palindrome-product/discuss/96294/could-any-python-experts-share-their-codes-within-100ms
        if n == 1:
            return 9
        for a in xrange(2, 9 * 10 ** (n - 1)):
            hi = (10 ** n) - a
            lo = int(str(hi)[::-1])
            if a ** 2 - 4 * lo < 0:
                continue
            if (a ** 2 - 4 * lo) ** .5 == int((a ** 2 - 4 * lo) ** .5):
                return (lo + 10 ** n * (10 ** n - a)) % 1337

# 0479 Hard 479 Largest Palindrome Product

In [ ]:
# Time:  O(n * 10^n)
# Space: O(n)

class Solution(object):
    def largestPalindrome(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 1:
            return 9
        # let x = 10^n-i, y = 10^n-j, s.t. palindrome = x*y
        # => (10^n-i)*(10^n-j) = (10^n-i-j)*10^n + i*j
        # assume i*j < 10^n (in fact, it only works for 2 <= n <= 8, not general),
        # let left = (10^n-i-j), right = i*j, k = i+j
        # => left = 10^n-k, right = i*(k-i)
        # => i^2 - k*i + right = 0
        # => i = (k+(k^2-right*4)^(0.5))/2 or (k+(k^2-right*4)^(0.5))/2 where i is a positive integer
        upper = 10**n-1
        for k in xrange(2, upper+1):
            left = 10**n-k
            right = int(str(left)[::-1])
            d = k**2-right*4
            if d < 0:
                continue
            if d**0.5 == int(d**0.5) and k%2 == int(d**0.5)%2:
                return (left*10**n+right)%1337
        return -1


# Time:  O(10^(2n))
# Space: O(n)
class Solution2(object):
    def largestPalindrome(self, n):
        """
        :type n: int
        :rtype: int
        """
        def divide_ceil(a, b):
            return (a+b-1)//b

        if n == 1:
            return 9
        upper, lower = 10**n-1, 10**(n-1)
        for i in reversed(xrange(lower, upper**2//(10**n)+1)):
            candidate = int(str(i) + str(i)[::-1])
            for y in reversed(xrange(divide_ceil(lower, 11)*11, upper+1, 11)):  # y must be divisible by 11 because even-number-length palindrome meets modulo 11 digit check
                if candidate//y > upper:
                    break
                if candidate%y == 0 and lower <= candidate//y:
                    return candidate%1337
        return -1

# 0480  Hard  480. Sliding Window Median

In [ ]:
class Solution:
    def medianSlidingWindow(self, nums, k):
        window = sorted(nums[:k])
        medians = []
        for a, b in zip(nums, nums[k:] + [0]):
            medians.append((window[k//2] + window[~(k//2)]) / 2.)
            window.remove(a)
            bisect.insort(window, b)
        return medians

# 0480 Hard 480 Sliding Window Median

In [ ]:
# Time:  O(nlogk)
# Space: O(k)

from sortedcontainers import SortedList


class Solution(object):
    def medianSlidingWindow(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: List[float]
        """
        sl = SortedList(float(nums[i])for i in xrange(k))
        result = [(sl[k//2]+sl[k//2-(1-k%2)])/2]
        for i in xrange(k, len(nums)):
            sl.add(float(nums[i]))
            sl.remove(nums[i-k])
            result.append((sl[k//2]+sl[k//2-(1-k%2)])/2)
        return result


# Time:  O(nlogk)
# Space: O(k)
import collections
import heapq


class Solution2(object):
    def medianSlidingWindow(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: List[float]
        """
        def lazy_delete(heap, to_remove, sign):
            while heap and sign*heap[0] in to_remove:
                to_remove[sign*heap[0]] -= 1
                if not to_remove[sign*heap[0]]:
                    del to_remove[sign*heap[0]]
                heapq.heappop(heap)

        def full_delete(heap, to_remove, sign):  # Time: O(k), Space: O(k)
            result = []
            for x in heap:
                if sign*x not in to_remove:
                    result.append(x)
                    continue
                to_remove[sign*x] -= 1
                if not to_remove[sign*x]:
                    del to_remove[sign*x]
            heap[:] = result
            heapify(heap)

        min_heap, max_heap = [], []
        for i in xrange(k):
            if i%2 == 0:
                heapq.heappush(min_heap, -heapq.heappushpop(max_heap, -nums[i]))
            else:
                heapq.heappush(max_heap, -heapq.heappushpop(min_heap, nums[i]))
        result = [float(min_heap[0])] if k%2 else [(min_heap[0]-max_heap[0])/2.0]
        to_remove = collections.defaultdict(int)
        for i in xrange(k, len(nums)):
            heapq.heappush(max_heap, -heapq.heappushpop(min_heap, nums[i]))
            if nums[i-k] > -max_heap[0]:
                heapq.heappush(min_heap, -heapq.heappop(max_heap))
            to_remove[nums[i-k]] += 1
            lazy_delete(max_heap, to_remove, -1)
            lazy_delete(min_heap, to_remove, 1)
            if len(min_heap)+len(max_heap) > 2*k:
                full_delete(max_heap, to_remove, -1)
                full_delete(min_heap, to_remove, 1)
            result.append(float(min_heap[0]) if k%2 else (min_heap[0]-max_heap[0])/2.0)
        return result


# Time:  O(nlogn) due to lazy delete
# Space: O(n)
import collections
import heapq


class Solution3(object):
    def medianSlidingWindow(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: List[float]
        """
        def lazy_delete(heap, to_remove, sign):
            while heap and sign*heap[0] in to_remove:
                to_remove[sign*heap[0]] -= 1
                if not to_remove[sign*heap[0]]:
                    del to_remove[sign*heap[0]]
                heapq.heappop(heap)

        min_heap, max_heap = [], []
        for i in xrange(k):
            if i%2 == 0:
                heapq.heappush(min_heap, -heapq.heappushpop(max_heap, -nums[i]))
            else:
                heapq.heappush(max_heap, -heapq.heappushpop(min_heap, nums[i]))
        result = [float(min_heap[0])] if k%2 else [(min_heap[0]-max_heap[0])/2.0]
        to_remove = collections.defaultdict(int)
        for i in xrange(k, len(nums)):
            heapq.heappush(max_heap, -heapq.heappushpop(min_heap, nums[i]))
            if nums[i-k] > -max_heap[0]:
                heapq.heappush(min_heap, -heapq.heappop(max_heap))
            to_remove[nums[i-k]] += 1
            lazy_delete(max_heap, to_remove, -1)
            lazy_delete(min_heap, to_remove, 1)
            result.append(float(min_heap[0]) if k%2 else (min_heap[0]-max_heap[0])/2.0)
        return result

# 0483  Hard  483. Smallest Good Base

In [ ]:
class Solution:
    def smallestGoodBase(self, n):
        n = int(n)
        for m in range(int(math.log(n, 2)), 1, -1):
            k = int(n ** m ** -1)
            if (k ** (m + 1) -1) // (k - 1) == n:
                return str(k)
        return str(n-1) 

# 0483 Hard 483 Smallest Good Base

In [ ]:
# Time:  O(logn * log(logn))
# Space: O(1)

import math


class Solution(object):
    def smallestGoodBase(self, n):
        """
        :type n: str
        :rtype: str
        """
        num = int(n)
        max_len = int(math.log(num,2))
        for l in xrange(max_len, 1, -1):
            b = int(num ** (l**-1))
            if (b**(l+1)-1) // (b-1) == num:
                return str(b)
        return str(num-1)

# 0488  Hard  488. Zuma Game

In [ ]:
class Solution:
    def findMinStep(self, board, hand):   
        def dfs(s, c):
            if not s: return 0
            res, i = float("inf"), 0
            while i < len(s):
                j = i + 1
                while j < len(s) and s[i] == s[j]: j += 1
                incr = 3 - (j - i)
                if c[s[i]] >= incr:
                    incr = 0 if incr < 0 else incr
                    c[s[i]] -= incr
                    tep = dfs(s[:i] + s[j:], c)
                    if tep >= 0: res = min(res, tep + incr)
                    c[s[i]] += incr
                i = j
            return res if res != float("inf") else -1
        return dfs(board, collections.Counter(hand))

# 0488 Hard 488 Zuma Game

In [ ]:
# Time:  O((b+h)^2 * h!*(b+h-1)!/(b-1)!)
# Space: O((b+h) * h!*(b+h-1)!/(b-1)!)

import collections


# brute force solution with worse complexity but pass
class Solution(object):
    def findMinStep(self, board, hand):
        """
        :type board: str
        :type hand: str
        :rtype: int
        """
        def shrink(s):  # Time: O(n^2), Space: O(1)
            while True:
                i = 0
                for start in xrange(len(s)):
                    while i < len(s) and s[start] == s[i]:
                        i += 1
                    if i-start >= 3:
                        s = s[0:start]+s[i:]
                        break
                else:
                    break
            return s

        def findMinStepHelper(board, hand, lookup):
            if not board: return 0
            if not hand: return float("inf")
            if tuple(hand) in lookup[tuple(board)]: return lookup[tuple(board)][tuple(hand)]

            result = float("inf")
            for i in xrange(len(hand)):
                for j in xrange(len(board)+1):
                    next_board = shrink(board[0:j] + hand[i:i+1] + board[j:])
                    next_hand = hand[0:i] + hand[i+1:]
                    result = min(result, findMinStepHelper(next_board, next_hand, lookup) + 1)
            lookup[tuple(board)][tuple(hand)] = result
            return result

        lookup = collections.defaultdict(dict)
        board, hand = list(board), list(hand)
        result = findMinStepHelper(board, hand, lookup)
        return -1 if result == float("inf") else result


# Time:  O((b+h) * h!*(b+h-1)!/(b-1)!)
# Space: O((b+h) * h!*(b+h-1)!/(b-1)!)
import collections


# brute force solution
class Solution_TLE(object):
    def findMinStep(self, board, hand):
        """
        :type board: str
        :type hand: str
        :rtype: int
        """
        def shrink(s):  # Time: O(n), Space: O(n)
            stack = []
            start = 0
            for i in xrange(len(s)+1):
                if i == len(s) or s[i] != s[start]:
                    if stack and stack[-1][0] == s[start]:
                        stack[-1][1] += i - start
                        if stack[-1][1] >= 3:
                            stack.pop()
                    elif s and i - start < 3:
                        stack += [s[start], i - start],
                    start = i
            result = []
            for p in stack:
                result += [p[0]] * p[1]
            return result

        def findMinStepHelper(board, hand, lookup):
            if not board: return 0
            if not hand: return float("inf")
            if tuple(hand) in lookup[tuple(board)]: return lookup[tuple(board)][tuple(hand)]

            result = float("inf")
            for i in xrange(len(hand)):
                for j in xrange(len(board)+1):
                    next_board = shrink(board[0:j] + hand[i:i+1] + board[j:])
                    next_hand = hand[0:i] + hand[i+1:]
                    result = min(result, findMinStepHelper(next_board, next_hand, lookup) + 1)
            lookup[tuple(board)][tuple(hand)] = result
            return result

        lookup = collections.defaultdict(dict)
        board, hand = list(board), list(hand)
        result = findMinStepHelper(board, hand, lookup)
        return -1 if result == float("inf") else result


# Time:  O((b * h) * b * b! * h!)
# Space: O(b * b! * h!)
import collections


# greedy solution without proof (possibly incorrect)
class Solution_GREEDY_ACCEPT_BUT_NOT_PROVED(object):
    def findMinStep(self, board, hand):
        """
        :type board: str
        :type hand: str
        :rtype: int
        """
        def shrink(s):  # Time: O(n), Space: O(n)
            stack = []
            start = 0
            for i in xrange(len(s)+1):
                if i == len(s) or s[i] != s[start]:
                    if stack and stack[-1][0] == s[start]:
                        stack[-1][1] += i - start
                        if stack[-1][1] >= 3:
                            stack.pop()
                    elif s and i - start < 3:
                        stack += [s[start], i - start],
                    start = i
            result = []
            for p in stack:
                result += [p[0]] * p[1]
            return result

        def findMinStepHelper2(board, hand, lookup):
            result = float("inf")
            for i in xrange(len(hand)):
                for j in xrange(len(board)+1):
                    next_board = shrink(board[0:j] + hand[i:i+1] + board[j:])
                    next_hand = hand[0:i] + hand[i+1:]
                    result = min(result, findMinStepHelper(next_board, next_hand, lookup) + 1)
            return result

        def find(board, c, j):
            for i in xrange(j, len(board)):
                if board[i] == c:
                    return i
            return -1

        def findMinStepHelper(board, hand, lookup):
            if not board: return 0
            if not hand: return float("inf")
            if tuple(hand) in lookup[tuple(board)]: return lookup[tuple(board)][tuple(hand)]

            result = float("inf")
            for i in xrange(len(hand)):
                j = 0
                while j < len(board):
                    k = find(board, hand[i], j)
                    if k == -1:
                        break

                    if k < len(board) - 1 and board[k] == board[k+1]:
                        next_board = shrink(board[0:k] + board[k+2:])
                        next_hand = hand[0:i] + hand[i+1:]
                        result = min(result, findMinStepHelper(next_board, next_hand, lookup) + 1)
                        k += 1
                    elif i > 0 and hand[i] == hand[i-1]:
                        next_board = shrink(board[0:k] + board[k+1:])
                        next_hand = hand[0:i-1] + hand[i+1:]
                        result = min(result, findMinStepHelper(next_board, next_hand, lookup) + 2)
                    j = k+1

            lookup[tuple(board)][tuple(hand)] = result
            return result
        
        board, hand = list(board), list(hand)
        hand.sort()
        result = findMinStepHelper(board, hand, collections.defaultdict(dict))
        if result == float("inf"):
            result = findMinStepHelper2(board, hand, collections.defaultdict(dict))
        return -1 if result == float("inf") else result


# Time:  O(b * b! * h!)
# Space: O(b * b! * h!)
# if a ball can be only inserted beside a ball with same color,
# we can do by this solution
class Solution_WRONG_GREEDY_AND_NOT_ACCEPT_NOW(object):
    def findMinStep(self, board, hand):
        """
        :type board: str
        :type hand: str
        :rtype: int
        """
        def shrink(s):  # Time: O(n), Space: O(n)
            stack = []
            start = 0
            for i in xrange(len(s)+1):
                if i == len(s) or s[i] != s[start]:
                    if stack and stack[-1][0] == s[start]:
                        stack[-1][1] += i - start
                        if stack[-1][1] >= 3:
                            stack.pop()
                    elif s and i - start < 3:
                        stack += [s[start], i - start],
                    start = i
            result = []
            for p in stack:
                result += [p[0]] * p[1]
            return result

        def find(board, c, j):
            for i in xrange(j, len(board)):
                if board[i] == c:
                    return i
            return -1

        def findMinStepHelper(board, hand, lookup):
            if not board: return 0
            if not hand: return float("inf")
            if tuple(hand) in lookup[tuple(board)]: return lookup[tuple(board)][tuple(hand)]

            result = float("inf")
            for i in xrange(len(hand)):
                j = 0
                while j < len(board):
                    k = find(board, hand[i], j)
                    if k == -1:
                        break

                    if k < len(board) - 1 and board[k] == board[k+1]:
                        next_board = shrink(board[0:k] + board[k+2:])
                        next_hand = hand[0:i] + hand[i+1:]
                        result = min(result, findMinStepHelper(next_board, next_hand, lookup) + 1)
                        k += 1
                    elif i > 0 and hand[i] == hand[i-1]:
                        next_board = shrink(board[0:k] + board[k+1:])
                        next_hand = hand[0:i-1] + hand[i+1:]
                        result = min(result, findMinStepHelper(next_board, next_hand, lookup) + 2)
                    j = k+1

            lookup[tuple(board)][tuple(hand)] = result
            return result

        lookup = collections.defaultdict(dict)
        board, hand = list(board), list(hand)
        hand.sort()
        result = findMinStepHelper(board, hand, lookup)
        return -1 if result == float("inf") else result

# 0489  Hard  489. Robot Room Cleaner

In [ ]:
class Solution:
    def cleanRoom(self, robot, move = [(-1, 0), (0, -1), (1, 0), (0, 1)]):
        def dfs(i, j, cleaned, ind):
            robot.clean()
            cleaned.add((i, j))
            k = 0
            for x, y in move[ind:] + move[:ind]:
                if (i + x, j + y) not in cleaned and robot.move():
                    dfs(i + x, j + y, cleaned, (ind + k) % 4)
                    robot.turnLeft()
                    robot.turnLeft()
                    robot.move()
                    robot.turnRight()
                    robot.turnRight()
                robot.turnLeft()
                k += 1
        dfs(0, 0, set(), 0) 

# 0489 Hard 489 Robot Room Cleaner

In [ ]:
# Time:  O(n), n is the number of cells
# Space: O(n)

class Solution(object):
    def cleanRoom(self, robot):
        """
        :type robot: Robot
        :rtype: None
        """
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

        def goBack(robot):
            robot.turnLeft()
            robot.turnLeft()
            robot.move()
            robot.turnRight()
            robot.turnRight()

        def dfs(pos, robot, d, lookup):
            robot.clean()
            for _ in directions:
                new_pos = (pos[0]+directions[d][0],
                           pos[1]+directions[d][1])
                if new_pos not in lookup:
                    lookup.add(new_pos)
                    if robot.move():
                        dfs(new_pos, robot, d, lookup)
                        goBack(robot)
                robot.turnRight()
                d = (d+1) % len(directions)
        
        dfs((0, 0), robot, 0, set())

# 0493  Hard  493. Reverse Pairs

In [ ]:
class Solution:
    def reversePairs(self, nums: List[int]) -> int:
        res = [0]
        def merge(nums):
            if len(nums) <= 1: return nums
            left, right = merge(nums[:len(nums)//2]), merge(nums[len(nums)//2:])
            for r in right:
                add = len(left) - bisect.bisect(left, 2 * r)
                if not add: break
                res[0] += add
            return sorted(left+right)
        merge(nums)
        return res[0]

# 0493 Hard 493 Reverse Pairs

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

class Solution(object):
    def reversePairs(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        def merge(nums, start, mid, end):
            r = mid + 1
            tmp = []
            for i in xrange(start, mid + 1):
                while r <= end and nums[i] > nums[r]:
                    tmp.append(nums[r])
                    r += 1
                tmp.append(nums[i])
            nums[start:start+len(tmp)] = tmp

        def countAndMergeSort(nums, start, end):
            if end - start <= 0:
                return 0

            mid = start + (end - start) / 2
            count = countAndMergeSort(nums, start, mid) + countAndMergeSort(nums, mid + 1, end)
            r = mid + 1
            for i in xrange(start, mid + 1):
                while r <= end and nums[i] > nums[r] * 2:
                    r += 1
                count += r - (mid + 1)
            merge(nums, start, mid, end)
            return count

        return countAndMergeSort(nums, 0, len(nums) - 1)

# 0499  Hard  499. The Maze III

In [ ]:
class Solution:
    def findShortestWay(self, maze, ball, hole):
        m, n, q, stopped = len(maze), len(maze[0]), [(0, "", ball[0], ball[1])], {(ball[0], ball[1]): [0, ""]}
        while q:
            dist, pattern, x, y = heapq.heappop(q)
            if [x, y] == hole:
                return pattern
            for i, j, p in ((-1, 0, "u"), (1, 0, "d"), (0, -1, "l"), (0, 1, "r")):
                newX, newY, d = x, y, 0
                while 0 <= newX + i < m and 0 <= newY + j < n and maze[newX + i][newY + j] != 1:
                    newX += i
                    newY += j
                    d += 1
                    if [newX, newY] == hole:
                        break
                if (newX, newY) not in stopped or [dist + d, pattern + p] < stopped[(newX, newY)]:
                    stopped[(newX, newY)] = [dist + d, pattern + p]
                    heapq.heappush(q, (dist + d, pattern + p, newX, newY))
        return "impossible"

# 0499 Hard 499 The Maze III

In [ ]:
# Time:  O(max(r, c) * wlogw)
# Space: O(w^2)

import heapq


class Solution(object):
    def findShortestWay(self, maze, ball, hole):
        """
        :type maze: List[List[int]]
        :type ball: List[int]
        :type hole: List[int]
        :rtype: str
        """
        ball, hole = tuple(ball), tuple(hole)
        dirs = {'u' : (-1, 0), 'r' : (0, 1), 'l' : (0, -1), 'd': (1, 0)}

        def neighbors(maze, node):
            for dir, vec in dirs.iteritems():
                cur_node, dist = list(node), 0
                while 0 <= cur_node[0]+vec[0] < len(maze) and \
                      0 <= cur_node[1]+vec[1] < len(maze[0]) and \
                      not maze[cur_node[0]+vec[0]][cur_node[1]+vec[1]]:
                    cur_node[0] += vec[0]
                    cur_node[1] += vec[1]
                    dist += 1
                    if tuple(cur_node) == hole:
                        break
                yield tuple(cur_node), dir, dist

        heap = [(0, '', ball)]
        visited = set()
        while heap:
            dist, path, node = heapq.heappop(heap)
            if node in visited: continue
            if node == hole: return path
            visited.add(node)
            for neighbor, dir, neighbor_dist in neighbors(maze, node):
                heapq.heappush(heap, (dist+neighbor_dist, path+dir, neighbor))

        return "impossible"

# 0502  Hard  502. IPO

In [ ]:
class Solution:
    def findMaximizedCapital(self, k, W, Profits, Capital):
        pool, new = [], [(c, p) for c, p in zip(Capital, Profits)]
        heapq.heapify(new)
        for i in range(k):
            while new and new[0][0] <= W:
                c, p = heapq.heappop(new)
                heapq.heappush(pool, -p)
            try:
                p = -heapq.heappop(pool)
                W += p
            except:
                break
        return W

# 0502 Hard 502 IPO

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import heapq


class Solution(object):
    def findMaximizedCapital(self, k, W, Profits, Capital):
        """
        :type k: int
        :type W: int
        :type Profits: List[int]
        :type Capital: List[int]
        :rtype: int
        """
        curr = []
        future = sorted(zip(Capital, Profits), reverse=True)
        for _ in xrange(k):
            while future and future[-1][0] <= W:
                heapq.heappush(curr, -future.pop()[1])
            if curr:
                W -= heapq.heappop(curr)
        return W

# 0514  Hard  514. Freedom Trail

In [ ]:
class Solution:
    def findRotateSteps(self, ring, key):
        ind, n, dp, pre = collections.defaultdict(list), len(ring), [0] * len(ring), key[0]
        for i, c in enumerate(ring): ind[c].append(i)
        for i in ind[key[0]]: dp[i] = min(i, n - i) + 1
        for c in key[1:]:
            for i in ind[c]: dp[i] = min(dp[j] + min(i - j, j + n - i) if i >= j else dp[j] + min(j - i, i + n - j) for j in ind[pre]) + 1
            pre = c
        return min(dp[i] for i in ind[key[-1]])

# 0514 Hard 514 Freedom Trail

In [ ]:
# Time:  O(k) ~ O(k * r^2)
# Space: O(r)

import collections


class Solution(object):
    def findRotateSteps(self, ring, key):
        """
        :type ring: str
        :type key: str
        :rtype: int
        """
        lookup = collections.defaultdict(list)
        for i in xrange(len(ring)):
            lookup[ring[i]].append(i)

        dp = [[0] * len(ring) for _ in xrange(2)]
        prev = [0]
        for i in xrange(1, len(key)+1):
            dp[i%2] = [float("inf")] * len(ring)
            for j in lookup[key[i-1]]:
                for k in prev:
                    dp[i%2][j] = min(dp[i%2][j],
                                     min((k+len(ring)-j) % len(ring), \
                                         (j+len(ring)-k) % len(ring)) + \
                                     dp[(i-1) % 2][k])
            prev = lookup[key[i-1]]
        return min(dp[len(key)%2]) + len(key)

# 0517  Hard  517. Super Washing Machines

In [ ]:
class Solution:
    def findMinMoves(self, machines):
        target, n, sm, res, total = sum(machines) // len(machines), len(machines), 0, 0, sum(machines)
        if target * n != total: return -1
        for i in range(n):
            l, sm, r = target * i - sm, sm + machines[i], target * (n - i - 1) - total + sm + machines[i]
            res = max(res, l + r, l, r)
        return res

# 0517 Hard 517 Super Washing Machines

In [ ]:
# Time:  O(n)
# Space: O(1)

class Solution(object):
    def findMinMoves(self, machines):
        """
        :type machines: List[int]
        :rtype: int
        """
        total = sum(machines)
        if total % len(machines): return -1

        result, target, curr = 0, total / len(machines), 0
        for n in machines:
            curr += n - target
            result = max(result, max(n - target, abs(curr)))
        return result

# 0527  Hard  527. Word Abbreviation

In [ ]:
class Solution:
    def wordsAbbreviation(self, dict):
        abb = collections.defaultdict(int)
        for i, w in enumerate(dict):
            for j in range(1, len(w) - 2):
                abb[w[:j] + str(len(w) - j - 1) + w[-1]] += 1
        for i, w in enumerate(dict):
            for j in range(1, len(w) - 2):
                new = w[:j] + str(len(w) - j - 1) + w[-1]
                if abb[new] == 1:
                    dict[i] = new
                    break
        return dict

# 0527 Hard 527 Word Abbreviation

In [ ]:
# Time:  O(n * l) ~ O(n^2 * l^2)
# Space: O(n * l)

import collections


class Solution(object):
    def wordsAbbreviation(self, dict):
        """
        :type dict: List[str]
        :rtype: List[str]
        """
        def isUnique(prefix, words):
            return sum(word.startswith(prefix) for word in words) == 1

        def toAbbr(prefix, word):
            abbr = prefix + str(len(word) - 1 - len(prefix)) + word[-1]
            return abbr if len(abbr) < len(word) else word

        abbr_to_word = collections.defaultdict(set)
        word_to_abbr = {}

        for word in dict:
            prefix = word[:1]
            abbr_to_word[toAbbr(prefix, word)].add(word)

        for abbr, conflicts in abbr_to_word.iteritems():
            if len(conflicts) > 1:
                for word in conflicts:
                    for i in xrange(2, len(word)):
                        prefix = word[:i]
                        if isUnique(prefix, conflicts):
                            word_to_abbr[word] = toAbbr(prefix, word)
                            break
            else:
                word_to_abbr[conflicts.pop()] = abbr

        return [word_to_abbr[word] for word in dict]

# 0546  Hard  546. Remove Boxes

In [ ]:
class Solution:
    def removeBoxes(self, A):
        n = len(A)
        memo = [[[0] * n for _ in range(n) ] for _ in range(n)]
        def dp(i, j, k):
            if i > j: return 0
            if not memo[i][j][k]:
                m = i
                while m+1 <= j and A[m+1] == A[i]:
                    m += 1
                i, k = m, k + m - i
                ans = dp(i+1, j, 0) + (k+1) ** 2
                for m in range(i+1, j+1):
                    if A[i] == A[m]:
                        ans = max(ans, dp(i+1, m-1, 0) + dp(m, j, k+1))
                memo[i][j][k] = ans
            return memo[i][j][k]
        return dp(0, n-1, 0)

# 0546 Hard 546 Remove Boxes

In [ ]:
# Time:  O(n^3) ~ O(n^4)
# Space: O(n^3)

class Solution(object):
    def removeBoxes(self, boxes):
        """
        :type boxes: List[int]
        :rtype: int
        """
        def dfs(boxes, l, r, k, lookup):
            if l > r: return 0
            if lookup[l][r][k]: return lookup[l][r][k]

            ll, kk = l, k
            while l < r and boxes[l+1] == boxes[l]:
                l += 1
                k += 1
            result = dfs(boxes, l+1, r, 0, lookup) + (k+1) ** 2
            for i in xrange(l+1, r+1):
                if boxes[i] == boxes[l]:
                    result = max(result, dfs(boxes, l+1, i-1, 0, lookup) + dfs(boxes, i, r, k+1, lookup))
            lookup[ll][r][kk] = result
            return result

        lookup = [[[0]*len(boxes) for _ in xrange(len(boxes)) ] for _ in xrange(len(boxes)) ]
        return dfs(boxes, 0, len(boxes)-1, 0, lookup)

# 0548  Hard  548. Split Array with Equal Sum

In [ ]:
class Solution:
    def splitArray(self, nums):
        n = len(nums)
        s = [0] * (n + 1)
        for i in range(n): s[i + 1] = s[i] + nums[i]
        def check(l, r):
            return set(s[m] - s[l] for m in range(l + 1, r + 1) if s[m] - s[l] == s[r + 1] - s[m + 1])
        return any(check(0, j - 1) & check(j + 1, n - 1) for j in range(n))

# 0548 Hard 548 Split Array with Equal Sum

In [ ]:
# Time:  O(n^2)
# Space: O(n)

class Solution(object):
    def splitArray(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        if len(nums) < 7:
            return False

        accumulated_sum = [0] * len(nums)
        accumulated_sum[0] = nums[0]
        for i in xrange(1, len(nums)):
            accumulated_sum[i] = accumulated_sum[i-1] + nums[i]
        for j in xrange(3, len(nums)-3):
            lookup = set()
            for i in xrange(1, j-1):
                if accumulated_sum[i-1] == accumulated_sum[j-1] - accumulated_sum[i]:
                    lookup.add(accumulated_sum[i-1])
            for k in xrange(j+2, len(nums)-1):
                if accumulated_sum[-1] - accumulated_sum[k] == accumulated_sum[k-1] - accumulated_sum[j] and \
                   accumulated_sum[k - 1] - accumulated_sum[j] in lookup:
                    return True
        return False

# 0552  Hard  552. Student Attendance Record II

In [ ]:
class Solution:
    def checkRecord(self, n):
        dp, mod = [1, 0, 0, 1, 1, 0], 10 ** 9 + 7
        for i in range(1, n):
            dp[0], dp[1], dp[2], dp[3], dp[4], dp[5] = sum(dp) % mod, dp[0], dp[1], sum(dp[3:]) % mod, dp[3], dp[4]
        return sum(dp) % mod

# 0552 Hard 552 Student Attendance Record II

In [ ]:
class Solution(object):
    def checkRecord(self, n):
        """
        dp[l] := number of eligible combination of a length l record without A
        """
        
        ans = 0
        M = 1000000007
        
        dp = [0]*max(n+1, 4)
        dp[0] = 1
        dp[1] = 2
        dp[2] = 4
        dp[3] = 7
        
        for i in xrange(4, n+1):
            dp[i] += dp[i-1]%M #ends at P
            dp[i] += (dp[i-1]%M - dp[i-4]%M) #ends at L. All posiblity but the end cannot be PLL
        
        ans += dp[n]
        
        for i in xrange(n):
            ans += dp[i] * dp[n-i-1]
            ans %= M
            
        return ans
        

# 0564  Hard  564. Find the Closest Palindrome

In [ ]:
class Solution:
    def nearestPalindromic(self, S):
        K = len(S)
        candidates = [str(10**k + d) for k in (K-1, K) for d in (-1, 1)]
        prefix = S[:(K+1)//2]
        P = int(prefix)
        for start in map(str, (P-1, P, P+1)):
            candidates.append(start + (start[:-1] if K%2 else start)[::-1])

        def delta(x):
            return abs(int(S) - int(x))

        ans = None
        for cand in candidates:
            if cand != S and not cand.startswith('00'):
                if (ans is None or delta(cand) < delta(ans) or
                        delta(cand) == delta(ans) and int(cand) < int(ans)):
                    ans = cand
        return ans

# 0564 Hard 564 Find the Closest Palindrome

In [ ]:
# Time:  O(l)
# Space: O(l)

class Solution(object):
    def nearestPalindromic(self, n):
        """
        :type n: str
        :rtype: str
        """
        l = len(n)
        candidates = set((str(10**l + 1), str(10**(l - 1) - 1)))
        prefix = int(n[:(l + 1)/2])
        for i in map(str, (prefix-1, prefix, prefix+1)):
            candidates.add(i + [i, i[:-1]][l%2][::-1])
        candidates.discard(n)
        return min(candidates, key=lambda x: (abs(int(x) - int(n)), int(x)))

# 0568  Hard  568. Maximum Vacation Days

In [ ]:
class Solution:
    def maxVacationDays(self, flights: List[List[int]], days: List[List[int]]) -> int:
        if not flights or not days:
            return 0
        n, k = len(flights), len(days[0])
        dp = [[-1] * (k + 1) for c in range(n)]
        dp[0][0] = 0
        for w in range(1, k + 1):
            for c in range(n):
                dp[c][w] = max([dp[pre][w - 1] + days[c][w - 1] for pre in range(n) if (flights[pre][c] or pre == c) and dp[pre][w - 1] >= 0] or [-1])
        return max(dp[c][-1] for c in range(n))

# 0568 Hard 568 Maximum Vacation Days

In [ ]:
# Time:  O(n^2 * k)
# Space: O(k)

class Solution(object):
    def maxVacationDays(self, flights, days):
        """
        :type flights: List[List[int]]
        :type days: List[List[int]]
        :rtype: int
        """
        if not days or not flights:
            return 0
        dp = [[0] * len(days) for _ in xrange(2)]
        for week in reversed(xrange(len(days[0]))):
            for cur_city in xrange(len(days)):
                dp[week % 2][cur_city] = days[cur_city][week] + dp[(week+1) % 2][cur_city]
                for dest_city in xrange(len(days)):
                    if flights[cur_city][dest_city] == 1:
                        dp[week % 2][cur_city] = max(dp[week % 2][cur_city], \
                                                     days[dest_city][week] + dp[(week+1) % 2][dest_city])
        return dp[0][0]

# 0587  Hard  587. Erect the Fence

In [ ]:
# http://www.algorithmist.com/index.php/Monotone_Chain_Convex_Hull.py


class Solution(object):

    def outerTrees(self, points):
        """Computes the convex hull of a set of 2D points.

        Input: an iterable sequence of (x, y) pairs representing the points.
        Output: a list of vertices of the convex hull in counter-clockwise order,
          starting from the vertex with the lexicographically smallest coordinates.
        Implements Andrew's monotone chain algorithm. O(n log n) complexity.
        """

        # Sort the points lexicographically (tuples are compared lexicographically).
        # Remove duplicates to detect the case we have just one unique point.
        # points = sorted(set(points))
        points = sorted(points, key=lambda p: (p.x, p.y))

        # Boring case: no points or a single point, possibly repeated multiple times.
        if len(points) <= 1:
            return points

        # 2D cross product of OA and OB vectors, i.e. z-component of their 3D cross product.
        # Returns a positive value, if OAB makes a counter-clockwise turn,
        # negative for clockwise turn, and zero if the points are collinear.
        def cross(o, a, b):
            # return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
            return (a.x - o.x) * (b.y - o.y) - (a.y - o.y) * (b.x - o.x)

        # Build lower hull
        lower = []
        for p in points:
            while len(lower) >= 2 and cross(lower[-2], lower[-1], p) < 0:
                lower.pop()
            lower.append(p)

        # Build upper hull
        upper = []
        for p in reversed(points):
            while len(upper) >= 2 and cross(upper[-2], upper[-1], p) < 0:
                upper.pop()
            upper.append(p)

        # Concatenation of the lower and upper hulls gives the convex hull.
        # Last point of each list is omitted because it is repeated at the
        # beginning of the other list.
        # return lower[:-1] + upper[:-1]
        return list(set(lower[:-1] + upper[:-1]))

# 0587 Hard 587 Erect the Fence

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

# Monotone Chain Algorithm
# Template: https://en.wikibooks.org/wiki/Algorithm_Implementation/Geometry/Convex_hull/Monotone_chain#Python
class Solution(object):
    def outerTrees(self, points):
        """
        :type points: List[List[int]]
        :rtype: List[List[int]]
        """
        # Sort the points lexicographically (tuples are compared lexicographically).
        # Remove duplicates to detect the case we have just one unique point.
        points = sorted(set(tuple(x) for x in points))

        # Boring case: no points or a single point, possibly repeated multiple times.
        if len(points) <= 1:
            return points

        # 2D cross product of OA and OB vectors, i.e. z-component of their 3D cross product.
        # Returns a positive value, if OAB makes a counter-clockwise turn,
        # negative for clockwise turn, and zero if the points are collinear.
        def cross(o, a, b):
            return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])

        # Build lower hull 
        lower = []
        for p in points:
            while len(lower) >= 2 and cross(lower[-2], lower[-1], p) < 0:  # modified
                lower.pop()
            lower.append(p)

        # Build upper hull
        upper = []
        for p in reversed(points):
            while len(upper) >= 2 and cross(upper[-2], upper[-1], p) < 0:  # modified
                upper.pop()
            upper.append(p)

        # Concatenation of the lower and upper hulls gives the convex hull.
        # Last point of each list is omitted because it is repeated at the beginning of the other list. 
        result = lower[:-1] + upper[:-1]
        return result if result[1] != result[-1] else result[:len(result)//2+1]  # modified

# 0588  Hard  588. Design In Memory File System

In [ ]:
class File:
    
    def __init__(self, name):
        self.name = name
        self.files = {}
        self.content = ''

class FileSystem:

    def __init__(self):
        self.root = File('/')

    def move(self, path):
        cur = self.root
        if path[1:]:
            for dr in path[1:].split('/'):
                if dr not in cur.files:
                    cur.files[dr] = File(dr)
                cur = cur.files[dr]
        return cur
    
    def ls(self, path: str) -> List[str]:
        cur = self.move(path)
        return [cur.name] if cur.content else sorted(cur.files.keys())

    def mkdir(self, path: str) -> None:
        self.move(path)

    def addContentToFile(self, filePath: str, content: str) -> None:
        cur = self.move(filePath)
        cur.content += content

    def readContentFromFile(self, filePath: str) -> str:
        return self.move(filePath).content
        


# Your FileSystem object will be instantiated and called as such:
# obj = FileSystem()
# param_1 = obj.ls(path)
# obj.mkdir(path)
# obj.addContentToFile(filePath,content)
# param_4 = obj.readContentFromFile(filePath)

# 0591  Hard  591. Tag Validator

In [ ]:
class Solution:
    def isValid(self, S):
        CDATA_BEGIN = '![CDATA['
        CDATA_END = ']]>'

        def collect_tag(i):
            for j in range(i, len(S)):
                if S[j] == '>': break
            else:
                return None
            return S[i+1:j]

        def valid_tag(tag):
            return 1 <= len(tag) <= 9 and all('A' <= c <= 'Z' for c in tag)

        if not S or S[0] != '<': return False
        tag = collect_tag(0)
        if (tag is None or 
                not S.startswith('<{}>'.format(tag)) or 
                not S.endswith('</{}>'.format(tag)) or
                not valid_tag(tag)):
            return False
        S = S[len(tag) + 2: -len(tag) - 3]

        i = 0
        stack = []
        while i < len(S):
            if S[i] == '<':
                tag = collect_tag(i)
                if tag is None: return False
                if tag.startswith(CDATA_BEGIN):
                    while i < len(S) and S[i:i+3] != CDATA_END:
                        i += 1
                    if not S[i:i+3] == CDATA_END:
                        return False
                    i += 2
                elif tag.startswith('/'):
                    tag = tag[1:]
                    if not valid_tag(tag) or not stack or stack.pop() != tag:
                        return False
                else:
                    if not valid_tag(tag):
                        return False
                    stack.append(tag)
            i += 1

        return not stack

# 0591 Hard 591 Tag Validator

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def isValid(self, code):
        """
        :type code: str
        :rtype: bool
        """
        def validText(s, i):
            j = i
            i = s.find("<", i)
            return i != j, i

        def validCData(s, i):
            if s.find("<![CDATA[", i) != i:
                return False, i
            j = s.find("]]>", i)
            if j == -1:
                return False, i
            return True, j+3

        def parseTagName(s, i):
            if s[i] != '<':
                return "", i
            j = s.find('>', i)
            if j == -1 or not (1 <= (j-1-i) <= 9):
                return "", i
            tag = s[i+1:j]
            for c in tag:
                if not (ord('A') <= ord(c) <= ord('Z')):
                    return "", i
            return tag, j+1

        def parseContent(s, i):
            while i < len(s):
                result, i = validText(s, i)
                if result:
                    continue
                result, i = validCData(s, i)
                if result:
                    continue
                result, i = validTag(s, i)
                if result:
                    continue
                break
            return i

        def validTag(s, i):
            tag, j = parseTagName(s, i)
            if not tag:
                return False, i
            j = parseContent(s, j)
            k = j + len(tag) + 2
            if k >= len(s) or s[j:k+1] != "</" + tag + ">":
                return False, i
            return True, k+1

        result, i = validTag(code, 0)
        return result and i == len(code)

# 0600  Hard  600. Non negative Integers without Consecutive Ones

In [ ]:
class Solution:
    def findIntegers(self, num):
        num, sub = bin(num)[2:], 0
        zero, one = [1] * len(num), [1] * len(num)
        for i in range(1, len(num)):
            zero[i], one[i] = zero[i - 1] + one[i - 1], zero[i - 1]
        for i in range(1, len(num)):
            if num[i] == num[i - 1] == "1": break
            if num[i] == num[i - 1] == "0": sub += one[-1 - i]
        return zero[-1] + one[-1] - sub

# 0629  Hard  629. K Inverse Pairs Array

In [ ]:
class Solution:
    def kInversePairs(self, n, k):
        dp = [1] + [0] * k
        for i in range(2, n + 1):
            for j in range(1, k + 1): dp[j] += dp[j - 1]
            for j in range(k, 0, -1): dp[j] -= j - i >= 0 and dp[j - i]
        return dp[-1] % (10 ** 9 + 7)

# 0629 Hard 629 K Inverse Pairs Array

In [ ]:
# Time:  O(n * k)
# Space: O(k)

class Solution(object):
    def kInversePairs(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: int
        """
        M = 1000000007
        dp = [[0]*(k+1) for _ in xrange(2)]
        dp[0][0] = 1
        for i in xrange(1, n+1):
            dp[i%2] = [0]*(k+1)
            dp[i%2][0] = 1
            for j in xrange(1, k+1):
                dp[i%2][j] = (dp[i%2][j-1] + dp[(i-1)%2][j]) % M
                if j-i >= 0:
                    dp[i%2][j] = (dp[i%2][j] - dp[(i-1)%2][j-i]) % M
        return dp[n%2][k]

# 0630  Hard  630. Course Schedule III

In [ ]:
class Solution:
    def scheduleCourse(self, courses):
        pq = []
        start = 0
        for t, end in sorted(courses, key = lambda x: x[1]):
            start += t
            heapq.heappush(pq, -t)
            while start > end:
                start += heapq.heappop(pq)
        return len(pq)

# 0630 Hard 630 Course Schedule III

In [ ]:
# Time:  O(nlogn)
# Space: O(k), k is the number of courses you can take

import collections
import heapq


class Solution(object):
    def scheduleCourse(self, courses):
        """
        :type courses: List[List[int]]
        :rtype: int
        """
        courses.sort(key=lambda t_end: t_end[1])
        max_heap = []
        now = 0
        for t, end in courses:
            now += t
            heapq.heappush(max_heap, -t)
            if now > end:
                now += heapq.heappop(max_heap)
        return len(max_heap)

# 0631  Hard  631. Design Excel Sum Formula

In [ ]:
class Excel(object):

    def __init__(self, H, W):
        self.M = [[{'v': 0, 'sum': None} for i in range(H)] for j in range(ord(W) - 64)]

    def set(self, r, c, v):
        self.M[r - 1][ord(c) - 65] = {'v': v, 'sum': None}

    def get(self, r, c):
        cell = self.M[r - 1][ord(c) - 65]
        if not cell['sum']: return cell['v']
        return sum(self.get(*pos) * cell['sum'][pos] for pos in cell['sum'])

    def sum(self, r, c, strs):
        self.M[r - 1][ord(c) - 65]['sum'] = self.parse(strs)
        return self.get(r, c)

    def parse(self, strs):
        c = collections.Counter()
        for s in strs:
            s, e = s.split(':')[0], s.split(':')[1] if ':' in s else s
            for i in range(int(s[1:]), int(e[1:]) + 1):
                for j in range(ord(s[0]) - 64, ord(e[0]) - 64 + 1):
                    c[(i, chr(j + 64))] += 1
        return c

# 0631 Hard 631 Design Excel Sum Formula

In [ ]:
# Time:  set: O((r * c)^2)
#        get: O(1)
#        sum: O((r * c)^2)
# Space: O(r * c)

import collections


class Excel(object):

    def __init__(self, H, W):
        """
        :type H: int
        :type W: str
        """
        self.__exl = [[0 for _ in xrange(ord(W)-ord('A')+1)] \
                      for _ in xrange(H+1)]
        self.__fward = collections.defaultdict(lambda : collections.defaultdict(int))
        self.__bward = collections.defaultdict(set)


    def set(self, r, c, v):
        """
        :type r: int
        :type c: str
        :type v: int
        :rtype: void
        """
        self.__reset_dependency(r, c)
        self.__update_others(r, c, v)


    def get(self, r, c):
        """
        :type r: int
        :type c: str
        :rtype: int
        """
        return self.__exl[r][ord(c) - ord('A')]


    def sum(self, r, c, strs):
        """
        :type r: int
        :type c: str
        :type strs: List[str]
        :rtype: int
        """
        self.__reset_dependency(r, c)
        result = self.__calc_and_update_dependency(r, c, strs)
        self.__update_others(r, c, result)
        return result


    def __reset_dependency(self, r, c):
        key = (r, c)
        if key in self.__bward.keys():
            for k in self.__bward[key]:
                self.__fward[k].pop(key, None)
            self.__bward[key] = set()


    def __calc_and_update_dependency(self, r, c, strs):
        result = 0
        for s in strs:
            s, e = s.split(':')[0], s.split(':')[1] if ':' in s else s
            left, right, top, bottom = ord(s[0])-ord('A'), ord(e[0])-ord('A'), int(s[1:]), int(e[1:])
            for i in xrange(top, bottom+1):
                for j in xrange(left, right+1):
                    result += self.__exl[i][j]
                    self.__fward[(i, chr(ord('A')+j))][(r, c)] += 1
                    self.__bward[(r, c)].add((i, chr(ord('A')+j)))
        return result


    def __update_others(self, r, c, v):
        prev = self.__exl[r][ord(c)-ord('A')]
        self.__exl[r][ord(c)-ord('A')] = v
        q = collections.deque()
        q.append(((r, c), v-prev))
        while q:
            key, diff = q.popleft()
            if key in self.__fward:
                for k, count in self.__fward[key].iteritems():
                    q.append((k, diff*count))
                    self.__exl[k[0]][ord(k[1])-ord('A')] += diff*count

# 0632  Hard  632. Smallest Range Covering Elements from K Lists

In [ ]:
class Solution:
    def smallestRange(self, nums):
        L = R = None
        while True:
            mn = mx = nums[0][-1]
            ind = [0]
            for i, ls in enumerate(nums[1:]):
                if ls[-1] > mx:
                    mx, ind = ls[-1], [i + 1]
                elif ls[-1] == mx:
                    ind.append(i + 1)
                elif ls[-1] < mn:
                    mn = ls[-1]
            if L == None or mx - mn <= R - L:
                L, R = mn, mx
            for j in ind:
                nums[j].pop()
                if not nums[j]:
                    return [L, R]

# 0639  Hard  639. Decode Ways II

In [ ]:
class Solution:
    def numDecodings(self, s):
        if s[0] == "0": return 0
        dp1 = dp2 = 1
        if s[0] == "*": dp2 = 9
        for i in range(1, len(s)):
            couple, newDp1 = s[i -1: i + 1], dp2
            if s[i] == "0":
                if s[i - 1] == "0" or s[i - 1] >= "3": return 0
                dp2 = 2 * dp1 if s[i - 1] == "*" else dp1
            elif s[i] == "*":
                dp2 *= 9
                if s[i - 1] == "2": dp2 += 6 * dp1
                elif s[i - 1] == "1": dp2 += 9 * dp1
                elif s[i - 1] == "*": dp2 += 15 * dp1
            elif "10" <= couple <= "26": dp2 += dp1
            elif s[i - 1] == "*": dp2 += 2 * dp1 if s[i] <= "6" else dp1
            dp1 = newDp1
        return dp2 % (10 ** 9 + 7)  

# 0639 Hard 639 Decode Ways II

In [ ]:
# Time:  O(n)
# Space: O(1)

class Solution(object):
    def numDecodings(self, s):
        """
        :type s: str
        :rtype: int
        """
        M, W = 1000000007, 3
        dp = [0] * W
        dp[0] = 1
        dp[1] = 9 if s[0] == '*' else dp[0] if s[0] != '0' else 0
        for i in xrange(1, len(s)):
            if s[i] == '*':
                dp[(i + 1) % W] = 9 * dp[i % W]
                if s[i - 1] == '1':
                    dp[(i + 1) % W] = (dp[(i + 1) % W] + 9 * dp[(i - 1) % W]) % M
                elif s[i - 1] == '2':
                    dp[(i + 1) % W] = (dp[(i + 1) % W] + 6 * dp[(i - 1) % W]) % M
                elif s[i - 1] == '*':
                    dp[(i + 1) % W] = (dp[(i + 1) % W] + 15 * dp[(i - 1) % W]) % M
            else:
                dp[(i + 1) % W] = dp[i % W] if s[i] != '0' else 0
                if s[i - 1] == '1':
                    dp[(i + 1) % W] = (dp[(i + 1) % W] + dp[(i - 1) % W]) % M
                elif s[i - 1] == '2' and s[i] <= '6':
                    dp[(i + 1) % W] = (dp[(i + 1) % W] + dp[(i - 1) % W]) % M
                elif s[i - 1] == '*':
                    dp[(i + 1) % W] = (dp[(i + 1) % W] + (2 if s[i] <= '6' else 1) * dp[(i - 1) % W]) % M
        return dp[len(s) % W]

# 0642  Hard  642. Design Search Autocomplete System

In [ ]:
class AutocompleteSystem:

    def __init__(self, sentences: List[str], times: List[int]):
        self.cur = self.root = {}
        self.rank = collections.defaultdict(int)
        for i, s in enumerate(sentences):
            self.s = s
            self.rank[s] = times[i] - 1
            self.input('#')
    
    def move(self, c):
        if c not in self.cur:
            self.cur[c] = {}
        self.cur = self.cur[c]
        if 'sentences' not in self.cur:
            self.cur['sentences'] = []
        
    def addSentence(self):
        self.cur = self.root
        for c in self.s:
            self.move(c)
            self.search()
            heapq.heappush(self.cur['sentences'], [-self.rank[self.s], self.s])
            
    def search(self):
        q, used, i = [], set(), 0
        while i < 3 and self.cur['sentences']:
            r, s = heapq.heappop(self.cur['sentences'])
            if s not in used:
                used.add(s)
                q.append([r, s])
                i += 1
        for r, s in q:
            heapq.heappush(self.cur['sentences'], [r, s])
        return [s for r, s in q]
            
    def input(self, c: str) -> List[str]:
        if c == '#':
            self.rank[self.s] += 1
            self.addSentence()
            self.s = ''
            self.cur = self.root
            return []
        else:
            self.s += c
            self.move(c)
            return self.search()
            


# Your AutocompleteSystem object will be instantiated and called as such:
# obj = AutocompleteSystem(sentences, times)
# param_1 = obj.input(c)

# 0642 Hard 642 Design Search Autocomplete System

In [ ]:
# Time:  O(p^2), p is the length of the prefix
# Space: O(p * t + s), t is the number of nodes of trie
#                    , s is the size of the sentences

import collections


class TrieNode(object):

    def __init__(self):
        self.__TOP_COUNT = 3
        self.infos = []
        self.leaves = {}


    def insert(self, s, times):
        cur = self
        cur.add_info(s, times)
        for c in s:
            if c not in cur.leaves:
                cur.leaves[c] = TrieNode()
            cur = cur.leaves[c]
            cur.add_info(s, times)


    def add_info(self, s, times):
        for p in self.infos:
            if p[1] == s:
                p[0] = -times
                break
        else:
            self.infos.append([-times, s])
        self.infos.sort()
        if len(self.infos) > self.__TOP_COUNT:
            self.infos.pop()


class AutocompleteSystem(object):

    def __init__(self, sentences, times):
        """
        :type sentences: List[str]
        :type times: List[int]
        """
        self.__trie = TrieNode()
        self.__cur_node = self.__trie
        self.__search = []
        self.__sentence_to_count = collections.defaultdict(int)
        for sentence, count in zip(sentences, times):
            self.__sentence_to_count[sentence] = count
            self.__trie.insert(sentence, count)


    def input(self, c):
        """
        :type c: str
        :rtype: List[str]
        """
        result = []
        if c == '#':
            self.__sentence_to_count["".join(self.__search)] += 1
            self.__trie.insert("".join(self.__search), self.__sentence_to_count["".join(self.__search)])
            self.__cur_node = self.__trie
            self.__search = []
        else:
            self.__search.append(c)
            if self.__cur_node:
                if c not in self.__cur_node.leaves:
                    self.__cur_node = None
                    return []
                self.__cur_node = self.__cur_node.leaves[c]
                result = [p[1] for p in self.__cur_node.infos]
        return result

# 0644  Hard  644. Maximum Average Subarray II

In [ ]:
class Solution:
    def findMaxAverage(self, nums: List[int], k: int) -> float:
        def sub(mid):
            sm = pre = mn = 0
            for i in range(k):
                sm += nums[i] - mid
            if sm >= 0:
                return True
            for i in range(k, len(nums)):
                sm += nums[i] - mid
                pre += nums[i - k] - mid
                mn = min(mn, pre)
                if sm >= mn:
                    return True
            return False
        l, r = min(nums), max(nums)
        while l + 1E-6 < r:
            mid = (l + r) / 2
            if sub(mid):
                l = mid
            else:
                r = mid
        return l

# 0644 Hard 644 Maximum Average Subarray II

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def findMaxAverage(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: float
        """
        def getDelta(avg, nums, k):
            accu = [0.0] * (len(nums) + 1)
            minval_pos = None
            delta = 0.0
            for i in xrange(len(nums)):
                accu[i+1] = nums[i] + accu[i] - avg
                if i >= (k-1):
                    if minval_pos == None or accu[i-k+1] < accu[minval_pos]:
                        minval_pos = i-k+1
                    if accu[i+1] - accu[minval_pos] >= 0:
                        delta = max(delta, (accu[i+1] - accu[minval_pos]) / (i+1 - minval_pos))
            return delta

        left, delta = min(nums), float("inf")
        while delta > 1e-5:
            delta = getDelta(left, nums, k)
            left += delta
        return left

# 0656  Hard  656. Coin Path

In [ ]:
class Solution:
    def cheapestJump(self, A, B): 
        n = len(A)
        preMin = {n - 1:[n]}
        for i in range(n - 2, -1, -1):
            if A[i] == -1:
                continue
            mn, preIndex = float("inf"), None
            for ind in range(i + 1, i + B + 1 <= n and i + B + 1 or n):
                if -1 < A[ind] < mn:
                    mn, preIndex = A[ind], ind
            if preIndex:
                A[i] += A[preIndex]
                preMin[i] = preMin[preIndex] + [i + 1]
            else:
                A[i] = -1
        return 0 in preMin and preMin[0][::-1] or []

# 0656 Hard 656 Coin Path

In [ ]:
# Time:  O(n * B)
# Space: O(n)

class Solution(object):
    def cheapestJump(self, A, B):
        """
        :type A: List[int]
        :type B: int
        :rtype: List[int]
        """
        result = []
        if not A or A[-1] == -1:
            return result
        n = len(A)
        dp, next_pos = [float("inf")] * n, [-1] * n
        dp[n-1] = A[n-1]
        for i in reversed(xrange(n-1)):
            if A[i] == -1:
                continue
            for j in xrange(i+1, min(i+B+1,n)):
                if A[i] + dp[j] < dp[i]:
                    dp[i] = A[i] + dp[j]
                    next_pos[i] = j
        if dp[0] == float("inf"):
            return result
        k = 0
        while k != -1:
            result.append(k+1)
            k = next_pos[k]
        return result

# 0660  Hard  660. Remove 9

In [ ]:
class Solution:
    def newInteger(self, n):
        base9 = ""
        while n:
            base9 += str(n % 9)
            n //= 9
        return int(base9[::-1])

# 0660 Hard 660 Remove 9

In [ ]:
# Time:  O(logn)
# Space: O(1)

class Solution(object):
    def newInteger(self, n):
        """
        :type n: int
        :rtype: int
        """
        result, base = 0, 1
        while n > 0:
            result += (n%9) * base
            n /= 9
            base *= 10
        return result

# 0664  Hard  664. Strange Printer

In [ ]:
class Solution:
    def strangePrinter(self, s):
        memo = {}
        def dp(i, j):
            if i > j: return 0
            if (i, j) not in memo:
                ans = dp(i+1, j) + 1
                for k in range(i+1, j+1):
                    if s[k] == s[i]:
                        ans = min(ans, dp(i, k-1) + dp(k+1, j))
                memo[i, j] = ans
            return memo[i, j]
        return dp(0, len(s) - 1)

# 0664 Hard 664 Strange Printer

In [ ]:
# Time:  O(n^3)
# Space: O(n^2)

class Solution(object):
    def strangePrinter(self, s):
        """
        :type s: str
        :rtype: int
        """
        def dp(s, i, j, lookup):
            if i > j:
                return 0
            if (i, j) not in lookup:
                lookup[(i, j)]  = dp(s, i, j-1, lookup) + 1
                for k in xrange(i, j):
                    if s[k] == s[j]:
                        lookup[(i, j)] = min(lookup[(i, j)], \
                                             dp(s, i, k, lookup) + dp(s, k+1, j-1, lookup))
            return lookup[(i, j)]

        lookup = {}
        return dp(s, 0, len(s)-1, lookup)

# 0668  Hard  668. Kth Smallest Number in Multiplication Table

In [ ]:
class Solution:
    def findKthNumber(self, m, n, k):       
        l, r = 1, m * n
        while l < r:
            mid = (l + r) // 2
            if sum(min(mid // i, n) for i in range(1, m + 1)) < k:
                l = mid + 1
            else:
                r = mid
        return l

# 0668 Hard 668. Kth Smallest Number in Multiplication Table

In [ ]:
class Solution:
    def findKthNumber(self, m: int, n: int, k: int) -> int:
        # https://leetcode.com/problems/kth-smallest-number-in-multiplication-table/solution/
        def enough(x):
            count = 0
            # ith row [i, 2*i, 3*i, ..., n*i]
            # for each column, k = x // i
            for i in range(1, m+1):
                count += min(x // i, n)
            return count >= k

        lo, hi = 1, m * n
        while lo < hi:
            mi = (lo + hi) // 2
            if not enough(mi):
                lo = mi + 1
            else:
                hi = mi
        return lo

# 0668 Hard 668 Kth Smallest Number in Multiplication Table

In [ ]:
# Time:  O(m * log(m * n))
# Space: O(1)

class Solution(object):
    def findKthNumber(self, m, n, k):
        """
        :type m: int
        :type n: int
        :type k: int
        :rtype: int
        """
        def count(target, m, n):
            return sum(min(target//i, n) for i in xrange(1, m+1))

        left, right = 1, m*n
        while left <= right:
            mid = left + (right-left)/2
            if count(mid, m, n) >= k:
                right = mid-1
            else:
                left = mid+1
        return left

# 0675  Hard  675. Cut Off Trees for Golf Event

In [ ]:
class Solution:
    def cutOffTree(self, forest):
        def hadlocks(forest, sr, sc, tr, tc):
            R, C = len(forest), len(forest[0])
            processed = set()
            deque = collections.deque([(0, sr, sc)])
            while deque:
                detours, r, c = deque.popleft()
                if (r, c) not in processed:
                    processed.add((r, c))
                    if r == tr and c == tc:
                        return abs(sr-tr) + abs(sc-tc) + 2*detours
                    for nr, nc, closer in ((r-1, c, r > tr), (r+1, c, r < tr),
                                           (r, c-1, c > tc), (r, c+1, c < tc)):
                        if 0 <= nr < R and 0 <= nc < C and forest[nr][nc]:
                            if closer:
                                deque.appendleft((detours, nr, nc))
                            else:
                                deque.append((detours+1, nr, nc))
            return -1
        trees = sorted((v, r, c) for r, row in enumerate(forest)
                       for c, v in enumerate(row) if v > 1)
        sr = sc = ans = 0
        for _, tr, tc in trees:
            d = hadlocks(forest, sr, sc, tr, tc)
            if d < 0: return -1
            ans += d
            sr, sc = tr, tc
        return ans

# 0675 Hard 675 Cut Off Trees for Golf Event

In [ ]:
# Time:  O(t * (logt + m * n)), t is the number of trees
# Space: O(t + m * n)

import collections
import heapq


class Solution(object):
    def cutOffTree(self, forest):
        """
        :type forest: List[List[int]]
        :rtype: int
        """
        def dot(p1, p2):
            return p1[0]*p2[0]+p1[1]*p2[1]

        def minStep(p1, p2):
            min_steps = abs(p1[0]-p2[0])+abs(p1[1]-p2[1])
            closer, detour = [p1], []
            lookup = set()
            while True:
                if not closer:  # cannot find a path in the closer expansions
                    if not detour:  # no other possible path
                        return -1
                    # try other possible paths in detour expansions with extra 2-step cost
                    min_steps += 2
                    closer, detour = detour, closer
                i, j = closer.pop()
                if (i, j) == p2:
                    return min_steps
                if (i, j) not in lookup:
                    lookup.add((i, j))
                    for I, J in (i+1, j), (i-1, j), (i, j+1), (i, j-1):
                        if 0 <= I < m and 0 <= J < n and forest[I][J] and (I, J) not in lookup:
                            is_closer = dot((I-i, J-j), (p2[0]-i, p2[1]-j)) > 0
                            (closer if is_closer else detour).append((I, J))
            return min_steps

        m, n = len(forest), len(forest[0])
        min_heap = []
        for i in xrange(m):
            for j in xrange(n):
                if forest[i][j] > 1:
                    heapq.heappush(min_heap, (forest[i][j], (i, j)))

        start = (0, 0)
        result = 0
        while min_heap:
            tree = heapq.heappop(min_heap)
            step = minStep(start, tree[1])
            if step < 0:
                return -1
            result += step
            start = tree[1]
        return result


# Time:  O(t * (logt + m * n)), t is the number of trees
# Space: O(t + m * n)
class Solution_TLE(object):
    def cutOffTree(self, forest):
        """
        :type forest: List[List[int]]
        :rtype: int
        """
        def minStep(p1, p2):
            min_steps = 0
            lookup = {p1}
            q = collections.deque([p1])
            while q:
                size = len(q)
                for _ in xrange(size):
                    (i, j) = q.popleft()
                    if (i, j) == p2:
                        return min_steps
                    for i, j in (i+1, j), (i-1, j), (i, j+1), (i, j-1):
                        if not (0 <= i < m and 0 <= j < n and forest[i][j] and (i, j) not in lookup):
                            continue
                        q.append((i, j))
                        lookup.add((i, j))
                min_steps += 1
            return -1

        m, n = len(forest), len(forest[0])
        min_heap = []
        for i in xrange(m):
            for j in xrange(n):
                if forest[i][j] > 1:
                    heapq.heappush(min_heap, (forest[i][j], (i, j)))

        start = (0, 0)
        result = 0
        while min_heap:
            tree = heapq.heappop(min_heap)
            step = minStep(start, tree[1])
            if step < 0:
                return -1
            result += step
            start = tree[1]
        return result

# 0679  Hard  679. 24 Game

In [ ]:
class Solution:
    def judgePoint24(self, nums):
        q = [[None, nums[i]] + nums[:i] + nums[i + 1:] for i in range(len(nums))]
        while q:
            new = []
            for group1, group2, *rest in q:
                if not rest and group1:
                    for res in (group1 + group2, group1 - group2, group1 * group2, group2 and group1 / group2): 
                        if 23.999 <= res <= 24.0001: return True
                if not rest and not group1 and 23.999 <= group2 <= 24.0001: return True
                for i in range(len(rest)):
                    for newGroup2 in (group2 + rest[i], group2 - rest[i], rest[i] - group2, group2 * rest[i], group2 / rest[i]):
                        new.append([group1, newGroup2] + rest[:i] + rest[i + 1:])
                    if group2:
                        new.append([group1, rest[i] / group2] + rest[:i] + rest[i + 1:])
                    if group1 != None:
                        for newGroup1 in (group1 + group2, group1 - group2, group1 * group2):
                            new.append([newGroup1, rest[i]] + rest[:i] + rest[i + 1:])
                        if group2:
                            new.append([group1 / group2, rest[i]] + rest[:i] + rest[i + 1:])
                    else:
                        new.append([group2, rest[i]] + rest[:i] + rest[i + 1:])
            q = new
        return False

# 0683  Hard  683. K Empty Slots

In [ ]:
class Node:
    def __init__(self, pos):
        self.pos = pos
        self.left = None
        self.right = None
        
class Solution(object):
    def kEmptySlots(self, flowers, k):
        """
        :type flowers: List[int]
        :type k: int
        :rtype: int
        """
        N = len(flowers)
        ans = -1
        nodes = {0:Node(-float('inf'))}
        for x in range(1, N + 2):
            nodes[x] = Node(x)
            nodes[x].left = nodes[x - 1]
            nodes[x - 1].right = nodes[x]
        nodes[N + 1].pos = float('inf')
        for day in range(N, 0, -1):
            x = flowers[day - 1]
            if nodes[x].pos - nodes[x].left.pos - 1 == k or nodes[x].right.pos - nodes[x].pos - 1 == k:
                ans = day
            nodes[x].left.right = nodes[x].right
            nodes[x].right.left = nodes[x].left
        return ans

# 0683 Hard 683 K Empty Slots

In [ ]:
"""
Let's look at the naive approach first.

Each day we turn on a bulb by `bulbs[i] = 1` and
1. `check()` the bulb on i to i+(K+1), see if it matches the pattern 1,0,0,..1
2. `check()` the bulb on i to i-(K+1), see if it matches the pattern 1,0,0,..1
The `check()` will take O(K), so the time complexity will be O(NK), N is the number of bulbs.

Since iterating the bulbs seems inevitable, the only thing we can avoid is the iteration between l and r in `check()` which is taking O(K) of time.
"""
class Solution(object):
    def kEmptySlots(self, schedule, K):
        def check(l, r):
            if l<0 or r>=len(bulbs): return False
            if bulbs[l]!=1 or bulbs[r]!=1: return False
            for k in xrange(l+1, r):
                if bulbs[k]!=0: return False
            return True
        
        bulbs = [0]*len(schedule)
        for day, x in enumerate(schedule):
            i = x-1
            bulbs[i] = 1
            if check(i, i+K+1): return day+1
            if check(i-(K+1), i): return day+1
        return -1


"""
The reason we iterate between l and r is because we need to check there are only "0" between l and r, no "1".
This is where we can use SortedSet (also known as TreeSet in Java).
For each day, we also add "1" index to the SortedSet.
So instead of checking if it is all "0" between l and r, we check if there is any "1" between l and r.

i = ss.bisect_right(l). The insetion point to a sorted list. If exist the same, insert to the right.
j = ss.bisect_left(r). The insetion point to a sorted list. If exist the same, insert to the left.

j==0 means that r is the smallest index in the SortedSet.
i==len(ss) means that l is the largest index in the SortedSet.
i==j means that there is no other "1" between l and r.

The time complexity is will be O(NLogN)
"""
from sortedcontainers import SortedSet

class Solution(object):
    def kEmptySlots(self, schedule, K):
        def check(l, r):
            if l<0 or r>=len(bulbs): return False
            if bulbs[l]!=1 or bulbs[r]!=1: return False
            
            i = ss.bisect_right(l)
            j = ss.bisect_left(r)
            
            return j!=0 and i!=len(ss) and i==j
        
        bulbs = [0]*len(schedule)
        ss = SortedSet()
        for day, x in enumerate(schedule):
            i = x-1
            bulbs[i] = 1
            ss.add(i)
            if check(i, i+K+1): return day+1
            if check(i-(K+1), i): return day+1
        return -1

# 0685  Hard  685. Redundant Connection II

In [ ]:
class Solution:
    def findRedundantDirectedConnection(self, edges):
        def root(i):
            return parent[i] == i and i or root(parent[i])
        
        parent, a, b, c = [0] * (len(edges) + 1), None, None, None
        for i, edge in enumerate(edges):
            if parent[edge[1]]:
                a, b, c, edges[i][0]= parent[edge[1]], edge[0], edge[1], 0
            else:
                parent[edge[1]] = edge[0]
        
        parent = [i for i in range(len(edges) + 1)]
        for u, v in edges:
            if u:
                if root(u) == v: 
                    return a and [a, c] or [u, v]
                parent[v] = u   
        return [b, c]

# 0685 Hard 685 Redundant Connection II

In [ ]:
# Time:  O(nlog*n) ~= O(n), n is the length of the positions
# Space: O(n)

class UnionFind(object):
    def __init__(self, n):
        self.set = range(n)

    def find_set(self, x):
        if self.set[x] != x:
            self.set[x] = self.find_set(self.set[x])  # path compression.
        return self.set[x]

    def union_set(self, x, y):
        x_root, y_root = map(self.find_set, (x, y))
        if x_root == y_root:
            return False
        self.set[min(x_root, y_root)] = max(x_root, y_root)
        return True


class Solution(object):
    def findRedundantDirectedConnection(self, edges):
        """
        :type edges: List[List[int]]
        :rtype: List[int]
        """
        cand1, cand2 = [], []
        parent = {}
        for edge in edges:
            if edge[1] not in parent:
                parent[edge[1]] = edge[0]
            else:
                cand1 = [parent[edge[1]], edge[1]]
                cand2 = edge

        union_find = UnionFind(len(edges)+1)
        for edge in edges:
            if edge == cand2:
                continue
            if not union_find.union_set(*edge):
                return cand1 if cand2 else edge
        return cand2

# 0689  Hard  689. Maximum Sum of 3 Non Overlapping Subarrays

In [ ]:
class Solution:
    def maxSumOfThreeSubarrays(self, nums, k):
        single, double, sm, n, cur = {}, {}, 0, len(nums), sum(nums[:k - 1])
        for i in range(k - 1, n):
            cur += nums[i]
            single[i - k + 1] = cur
            cur -= nums[i - k + 1]
        cur = n - k, single[n - k]
        for i in range(n - k, k * 2 - 1, -1):
            if single[i] >= cur[1]:
                cur = i, single[i]
            double[i - k] = cur[1] + single[i - k], i - k, cur[0]
        cur = double[n - 2 * k]
        for i in range(n - 2 * k, k - 1, -1):
            if double[i][0] >= cur[0]:
                cur = double[i]
            if single[i - k] + cur[0] >= sm:
                sm, res = single[i - k] + cur[0], [i - k, cur[1], cur[2]]
        return res

# 0691  Hard  691. Stickers to Spell Word

In [ ]:
class Solution:
    def minStickers(self, stickers, target):
        cnt, res, n = collections.Counter(target), [float("inf")], len(target)  
        def dfs(dic, used, i):
            if i == n:
                res[0] = min(res[0], used)
            elif dic[target[i]] >= cnt[target[i]]:
                dfs(dic, used, i + 1)
            elif used < res[0] - 1:
                for sticker in stickers:
                    if target[i] in sticker:
                        for s in sticker:
                            dic[s] += 1
                        dfs(dic, used + 1, i + 1)
                        for s in sticker:
                            dic[s] -= 1
        dfs(collections.defaultdict(int), 0, 0)
        return res[0] < float("inf") and res[0] or -1

# 0691 Hard 691 Stickers to Spell Word

In [ ]:
# Time:  O(T * S^T)
# Space: O(T * S^T)

import collections


class Solution(object):
    def minStickers(self, stickers, target):
        """
        :type stickers: List[str]
        :type target: str
        :rtype: int
        """
        def minStickersHelper(sticker_counts, target, dp):
            if "".join(target) in dp:
                return dp["".join(target)]
            target_count = collections.Counter(target)
            result = float("inf")
            for sticker_count in sticker_counts:
                if sticker_count[target[0]] == 0:
                    continue
                new_target = []
                for k in target_count.keys():
                    if target_count[k] > sticker_count[k]:
                       new_target += [k]*(target_count[k] - sticker_count[k])
                if len(new_target) != len(target):
                    num = minStickersHelper(sticker_counts, new_target, dp)
                    if num != -1:
                        result = min(result, 1+num)
            dp["".join(target)] = -1 if result == float("inf") else result
            return dp["".join(target)]

        sticker_counts = map(collections.Counter, stickers)
        dp = { "":0 }
        return minStickersHelper(sticker_counts, target, dp)

# 0699  Hard  699. Falling Squares

In [ ]:
class Solution:
    def fallingSquares(self, positions: List[List[int]]) -> List[int]:
        height = [0]
        pos = [0]
        res = []
        max_h = 0
        for left, side in positions:
            i = bisect.bisect_right(pos, left)
            j = bisect.bisect_left(pos, left + side)
            high = max(height[i - 1:j] or [0]) + side
            pos[i:j] = [left, left + side]
            height[i:j] = [high, height[j - 1]]
            max_h = max(max_h, high)
            res.append(max_h)
        return res

# 0699 Hard 699 Falling Squares

In [ ]:
# Time:  O(n^2), could be improved to O(nlogn) in cpp by ordered map (bst)
# Space: O(n)

import bisect


class Solution(object):
    def fallingSquares(self, positions):
        result = []
        pos = [-1]
        heights = [0]
        maxH = 0
        for left, side in positions:
            l = bisect.bisect_right(pos, left)
            r = bisect.bisect_left(pos, left+side)
            high = max(heights[l-1:r] or [0]) + side
            pos[l:r] = [left, left+side]         # Time: O(n)
            heights[l:r] = [high, heights[r-1]]  # Time: O(n)
            maxH = max(maxH, high)
            result.append(maxH)
        return result


class SegmentTree(object):
    def __init__(self, N,
                 query_fn=min,
                 update_fn=lambda x, y: y,
                 default_val=float("inf")):
        self.N = N
        self.H = (N-1).bit_length()
        self.query_fn = query_fn
        self.update_fn = update_fn
        self.default_val = default_val
        self.tree = [default_val] * (2 * N)
        self.lazy = [None] * N

    def __apply(self, x, val):
        self.tree[x] = self.update_fn(self.tree[x], val)
        if x < self.N:
            self.lazy[x] = self.update_fn(self.lazy[x], val)

    def update(self, L, R, h):
        def pull(x):
            while x > 1:
                x //= 2
                self.tree[x] = self.query_fn(self.tree[x*2], self.tree[x*2 + 1])
                if self.lazy[x] is not None:
                    self.tree[x] = self.update_fn(self.tree[x], self.lazy[x])

        L += self.N
        R += self.N
        L0, R0 = L, R
        while L <= R:
            if L & 1:
                self.__apply(L, h)
                L += 1
            if R & 1 == 0:
                self.__apply(R, h)
                R -= 1
            L //= 2
            R //= 2
        pull(L0)
        pull(R0)

    def query(self, L, R):
        def push(x):
            n = 2**self.H
            while n != 1:
                y = x // n
                if self.lazy[y] is not None:
                    self.__apply(y*2, self.lazy[y])
                    self.__apply(y*2 + 1, self.lazy[y])
                    self.lazy[y] = None
                n //= 2

        result = self.default_val
        if L > R:
            return result

        L += self.N
        R += self.N
        push(L)
        push(R)
        while L <= R:
            if L & 1:
                result = self.query_fn(result, self.tree[L])
                L += 1
            if R & 1 == 0:
                result = self.query_fn(result, self.tree[R])
                R -= 1
            L //= 2
            R //= 2
        return result
    
    def data(self):
        showList = []
        for i in xrange(self.N):
            showList.append(self.query(i, i))
        return showList


class SegmentTree2(object):
    def __init__(self, nums,
                 query_fn=min,
                 update_fn=lambda x, y: y,
                 default_val=float("inf")):
        """
        initialize your data structure here.
        :type nums: List[int]
        """
        N = len(nums)
        self.__original_length = N
        self.__tree_length = 2**(N.bit_length() + (N&(N-1) != 0))-1
        self.__query_fn = query_fn
        self.__update_fn = update_fn
        self.__default_val = default_val
        self.__tree = [default_val for _ in range(self.__tree_length)]
        self.__lazy = [None for _ in range(self.__tree_length)]
        self.__constructTree(nums, 0, self.__original_length-1, 0)

    def update(self, i, j, val):
        self.__updateTree(val, i, j, 0, self.__original_length-1, 0)

    def query(self, i, j):
        return self.__queryRange(i, j, 0, self.__original_length-1, 0)

    def __constructTree(self, nums, left, right, idx):
        if left > right:
             return
        if left == right:
            self.__tree[idx] = self.__update_fn(self.__tree[idx], nums[left])
            return 
        mid = left + (right-left)//2
        self.__constructTree(nums, left, mid, idx*2 + 1)
        self.__constructTree(nums, mid+1, right, idx*2 + 2)
        self.__tree[idx] = self.__query_fn(self.__tree[idx*2 + 1], self.__tree[idx*2 + 2])

    def __apply(self, left, right, idx, val):
        self.__tree[idx] = self.__update_fn(self.__tree[idx], val)
        if left != right:
            self.__lazy[idx*2 + 1] = self.__update_fn(self.__lazy[idx*2 + 1], val)
            self.__lazy[idx*2 + 2] = self.__update_fn(self.__lazy[idx*2 + 2], val)

    def __updateTree(self, val, range_left, range_right, left, right, idx):
        if left > right:
            return
        if self.__lazy[idx] is not None:
            self.__apply(left, right, idx, self.__lazy[idx])
            self.__lazy[idx] = None
        if range_left > right or range_right < left:
            return
        if range_left <= left and right <= range_right:
            self.__apply(left, right, idx, val)
            return
        mid = left + (right-left)//2
        self.__updateTree(val, range_left, range_right, left, mid, idx*2 + 1)
        self.__updateTree(val, range_left, range_right, mid+1, right, idx*2 + 2)
        self.__tree[idx] = self.__query_fn(self.__tree[idx*2 + 1],
                                           self.__tree[idx*2 + 2])

    def __queryRange(self, range_left, range_right, left, right, idx):
        if left > right:
            return self.__default_val
        if self.__lazy[idx] is not None:
            self.__apply(left, right, idx, self.__lazy[idx])
            self.__lazy[idx] = None
        if right < range_left or left > range_right:
            return self.__default_val
        if range_left <= left and right <= range_right:
            return self.__tree[idx]
        mid = left + (right-left)//2
        return self.__query_fn(self.__queryRange(range_left, range_right, left, mid, idx*2 + 1), 
                               self.__queryRange(range_left, range_right, mid + 1, right, idx*2 + 2))


# Time:  O(nlogn)
# Space: O(n)
# Segment Tree solution.
class Solution2(object):
    def fallingSquares(self, positions):
        index = set()
        for left, size in positions:
            index.add(left)
            index.add(left+size-1)
        index = sorted(list(index))
        tree = SegmentTree(len(index), max, max, 0)
        # tree = SegmentTree2([0]*len(index), max, max, 0)
        max_height = 0
        result = []
        for left, size in positions:
            L, R = bisect.bisect_left(index, left), bisect.bisect_left(index, left+size-1)
            h = tree.query(L, R) + size
            tree.update(L, R, h)
            max_height = max(max_height, h)
            result.append(max_height)
        return result


# Time:  O(n * sqrt(n))
# Space: O(n)
class Solution3(object):
    def fallingSquares(self, positions):
        def query(heights, left, right, B, blocks, blocks_read):
            result = 0
            while left % B and left <= right:
                result = max(result, heights[left], blocks[left//B])
                left += 1
            while right % B != B-1 and left <= right:
                result = max(result, heights[right], blocks[right//B])
                right -= 1
            while left <= right:
                result = max(result, blocks[left//B], blocks_read[left//B])
                left += B
            return result

        def update(heights, left, right, B, blocks, blocks_read, h):
            while left % B and left <= right:
                heights[left] = max(heights[left], h)
                blocks_read[left//B] = max(blocks_read[left//B], h)
                left += 1
            while right % B != B-1 and left <= right:
                heights[right] = max(heights[right], h)
                blocks_read[right//B] = max(blocks_read[right//B], h)
                right -= 1
            while left <= right:
                blocks[left//B] = max(blocks[left//B], h)
                left += B

        index = set()
        for left, size in positions:
            index.add(left)
            index.add(left+size-1)
        index = sorted(list(index))
        W = len(index)
        B = int(W**.5)
        heights = [0] * W
        blocks = [0] * (B+2)
        blocks_read = [0] * (B+2)

        max_height = 0
        result = []
        for left, size in positions:
            L, R = bisect.bisect_left(index, left), bisect.bisect_left(index, left+size-1)
            h = query(heights, L, R, B, blocks, blocks_read) + size
            update(heights, L, R, B, blocks, blocks_read, h)
            max_height = max(max_height, h)
            result.append(max_height)
        return result


# Time:  O(n^2)
# Space: O(n)
class Solution4(object):
    def fallingSquares(self, positions):
        """
        :type positions: List[List[int]]
        :rtype: List[int]
        """
        heights = [0] * len(positions)
        for i in xrange(len(positions)):
            left_i, size_i = positions[i]
            right_i = left_i + size_i
            heights[i] += size_i
            for j in xrange(i+1, len(positions)):
                left_j, size_j = positions[j]
                right_j = left_j + size_j
                if left_j < right_i and left_i < right_j:  # intersect
                    heights[j] = max(heights[j], heights[i])

        result = []
        for height in heights:
            result.append(max(result[-1], height) if result else height)
        return result

# 0710  Hard  710. Random Pick with Blacklist

In [ ]:
class Solution:

    def __init__(self, N, blacklist):
        self.forbidden, self.n, self.used, self.cur = set(blacklist), N, set(), 0
    def pick(self):
        while self.cur in self.forbidden: self.cur += 1
        if self.cur < self.n: num, self.cur = self.cur, self.cur + 1
        else: num = self.used.pop()
        self.used.add(num)
        return num

# 0710 Hard 710 Random Pick with Blacklist

In [ ]:
# Time:  ctor: O(b)
#        pick: O(1)
# Space: O(b)

import random


class Solution(object):
    
    def __init__(self, N, blacklist):
        """
        :type N: int
        :type blacklist: List[int]
        """
        self.__n = N-len(blacklist)
        self.__lookup = {}
        white = iter(set(range(self.__n, N))-set(blacklist))
        for black in blacklist:
            if black < self.__n:
                self.__lookup[black] = next(white)
        
        
    def pick(self):
        """
        :rtype: int
        """
        index = random.randint(0, self.__n-1)
        return self.__lookup[index] if index in self.__lookup else index


# Time:  ctor: O(blogb)
#        pick: O(logb)
# Space: O(b)
import random

class Solution2(object):
    
    def __init__(self, N, blacklist):
        """
        :type N: int
        :type blacklist: List[int]
        """
        self.__n = N-len(blacklist)
        blacklist.sort()
        self.__blacklist = blacklist
        
    def pick(self):
        """
        :rtype: int
        """
        index = random.randint(0, self.__n-1)
        left, right = 0, len(self.__blacklist)-1
        while left <= right:
            mid = left+(right-left)//2
            if index+mid < self.__blacklist[mid]:
                right = mid-1
            else:
                left = mid+1
        return index+left

# 0711  Hard  711. Number of Distinct Islands II

In [ ]:
class Solution:
    def numDistinctIslands2(self, grid: List[List[int]]) -> int:
        if not grid or not grid[0]: return 0
        m,n=len(grid),len(grid[0])

        # augment matrix to void length check
        grid.append([0]*n)
        for row in grid: row.append(0)

        self.pool=set()
        self.res=0

        def bfs(i0,j0):
            grid[i0][j0]=-1
            q=[(i0,j0)]
            for i,j in q:
                for I,J in (i-1,j),(i+1,j),(i,j-1),(i,j+1):
                    if grid[I][J]==1:
                        grid[I][J]=-1
                        q.append([I,J])
            self.addisland(q)
       
        for i in range(m):
            for j in range(n):
                if grid[i][j]==1: bfs(i,j)

        return self.res

    def addisland(self,q):
            Imin=min(x for x,y in q)
            Jmin=min(y for x,y in q)
            island1=tuple(sorted((x-Imin,y-Jmin) for x,y in q)) # original island
           
            if island1 in self.pool: return None
            self.res+=1

            Imax=max(x for x,y in island1)
            Jmax=max(y for x,y in island1)

            island2=tuple(sorted((-x+Imax,y) for x,y in island1)) # x axis mirror
            island3=tuple(sorted((x,-y+Jmax) for x,y in island1)) # y axis mirror
            island4=tuple(sorted((-x+Imax,-y+Jmax) for x,y in island1)) # origin mirror

            island5=tuple(sorted((y,x) for x,y in island1)) # diagonal mirror
            island6=tuple(sorted((-x+Jmax,y) for x,y in island5))
            island7=tuple(sorted((x,-y+Imax) for x,y in island5))
            island8=tuple(sorted((-x+Jmax,-y+Imax) for x,y in island5))

            self.pool |= set([island1,island2,island3,island4,island5,island6,island7,island8])

# 0711 Hard 711 Number of Distinct Islands II

In [ ]:
# Time:  O((m * n) * log(m * n))
# Space: O(m * n)

class Solution(object):
    def numDistinctIslands2(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]

        def dfs(i, j, grid, island):
            if not (0 <= i < len(grid) and \
                    0 <= j < len(grid[0]) and \
                    grid[i][j] > 0):
                return False
            grid[i][j] *= -1
            island.append((i, j))
            for d in directions:
                dfs(i+d[0], j+d[1], grid, island)
            return True

        def normalize(island):
            shapes = [[] for _ in xrange(8)]
            for x, y in island:
                rotations_and_reflections = [[ x,  y], [ x, -y], [-x, y], [-x, -y],
                                             [ y,  x], [ y, -x], [-y, x], [-y, -x]]
                for i in xrange(len(rotations_and_reflections)):
                    shapes[i].append(rotations_and_reflections[i])
            for shape in shapes:
                shape.sort()  # Time: O(ilogi), i is the size of the island, the max would be (m * n)
                origin = list(shape[0])
                for p in shape:
                    p[0] -= origin[0]
                    p[1] -= origin[1]
            return min(shapes)

        islands = set()
        for i in xrange(len(grid)):
            for j in xrange(len(grid[0])):
                island = []
                if dfs(i, j, grid, island):
                    islands.add(str(normalize(island)))
        return len(islands)

# 0715  Hard  715. Range Module

In [ ]:
from bisect import bisect_left as bl, bisect_right as br

class RangeModule:

    def __init__(self):
        self._X = []

    def addRange(self, left, right):
        i, j = bl(self._X, left), br(self._X, right)
        self._X[i:j] = [left]*(i%2 == 0) + [right]*(j%2 == 0)

    def queryRange(self, left, right):
        i, j = br(self._X, left), bl(self._X, right)
        return i == j and i%2 == 1

    def removeRange(self, left, right):
        i, j = bl(self._X, left), br(self._X, right)
        self._X[i:j] = [left]*(i%2 == 1) + [right]*(j%2 == 1)

# 0715 Hard 715 Range Module

In [ ]:
# Time:  addRange:    O(n)
#        removeRange: O(n)
#        queryRange:  O(logn)
# Space: O(n)

import bisect


class RangeModule(object):

    def __init__(self):
        self.__intervals = []

    def addRange(self, left, right):
        """
        :type left: int
        :type right: int
        :rtype: void
        """
        tmp = []
        i = 0
        for interval in self.__intervals:
            if right < interval[0]:
                tmp.append((left, right))
                break
            elif interval[1] < left:
                tmp.append(interval)
            else:
                left = min(left, interval[0])
                right = max(right, interval[1])
            i += 1
        if i == len(self.__intervals):
            tmp.append((left, right))
        while i < len(self.__intervals):
            tmp.append(self.__intervals[i])
            i += 1
        self.__intervals = tmp

    def queryRange(self, left, right):
        """
        :type left: int
        :type right: int
        :rtype: bool
        """
        i = bisect.bisect_left(self.__intervals, (left, float("inf")))
        if i: i -= 1
        return bool(self.__intervals) and \
               self.__intervals[i][0] <= left and \
               right <= self.__intervals[i][1]

    def removeRange(self, left, right):
        """
        :type left: int
        :type right: int
        :rtype: void
        """
        tmp = []
        for interval in self.__intervals:
            if interval[1] <= left or interval[0] >= right:
                tmp.append(interval)
            else:
                if interval[0] < left:
                    tmp.append((interval[0], left))
                if right < interval[1]:
                    tmp.append((right, interval[1]))
        self.__intervals = tmp

# 0716  Hard  716. Max Stack

In [ ]:
class Node:
    def __init__(self, value, index):
        self.val = value
        self.i = index
        self.pre = self.next = None
class MaxStack:

    def __init__(self):
        """
        initialize your data structure here.
        """
        self.heap = []
        self.Nodes = {}
        self.head = self.tail = Node(0, -1)
        self.head.next = self.tail
        self.tail.pre = self.head
        

    def push(self, x):
        """
        :type x: int
        :rtype: void
        """
        newNode = Node(x, self.tail.pre.i + 1)
        newNode.pre = self.tail.pre
        newNode.next = self.tail
        self.tail.pre.next = self.tail.pre = newNode
        self.Nodes[newNode.i] = newNode
        heapq.heappush(self.heap, (-x, -newNode.i))

    def pop(self):
        """
        :rtype: int
        """
        node = self.tail.pre
        node.pre.next = self.tail
        self.tail.pre = node.pre
        self.Nodes.pop(node.i)
        if node.i == -self.heap[0][1]:
            heapq.heappop(self.heap)
        return node.val

    def top(self):
        """
        :rtype: int
        """
        return self.tail.pre.val

    def peekMax(self):
        """
        :rtype: int
        """
        while -self.heap[0][1] not in self.Nodes or self.Nodes[-self.heap[0][1]].val != -self.heap[0][0]:
            heapq.heappop(self.heap)
        return -self.heap[0][0]

    def popMax(self):
        """
        :rtype: int
        """
        while -self.heap[0][1] not in self.Nodes or self.Nodes[-self.heap[0][1]].val != -self.heap[0][0]:
            heapq.heappop(self.heap)
        node = self.Nodes.pop(-self.heap[0][1])
        node.pre.next = node.next
        node.next.pre = node.pre
        return -heapq.heappop(self.heap)[0]


# Your MaxStack object will be instantiated and called as such:
# obj = MaxStack()
# obj.push(x)
# param_2 = obj.pop()
# param_3 = obj.top()
# param_4 = obj.peekMax()
# param_5 = obj.popMax()

# 0716 Hard 716. Max Stack

In [ ]:
class MaxStack(object):

    def __init__(self):
        """
        initialize your data structure here.
        """
        self.stack = []
        self.max_stack = []

    def push(self, x):
        """
        :type x: int
        :rtype: void
        """
        self.stack.append(x)
        if len(self.max_stack) == 0:
            self.max_stack.append(x)
            return
        if self.max_stack[-1] > x:
            self.max_stack.append(self.max_stack[-1])
        else:
            self.max_stack.append(x)

    def pop(self):
        """
        :rtype: int
        """
        if len(self.stack) != 0:
            self.max_stack.pop(-1)
            return self.stack.pop(-1)

    def top(self):
        """
        :rtype: int
        """
        return self.stack[-1]

    def peekMax(self):
        """
        :rtype: int
        """
        if len(self.max_stack) != 0:
            return self.max_stack[-1]

    def popMax(self):
        """
        :rtype: int
        """
        val = self.peekMax()
        buff = []
        while self.top() != val:
            buff.append(self.pop())
        self.pop()
        while len(buff) != 0:
            self.push(buff.pop(-1))
        return val

# 0716 Hard 716 Max Stack

In [ ]:
#https://leetcode.com/problems/max-stack/

#peekMax() and popMax() are O(N)
#Else are O(1)
class MaxStack(object):

    def __init__(self):
        self.stack = []
        

    def push(self, x):
        self.stack.append(x)
        

    def pop(self):
        return self.stack.pop()
        
        
    def top(self):
        return self.stack[-1]
        

    def peekMax(self):
        return max(self.stack)
        

    def popMax(self):
        max_num = None
        max_index = None
        
        for i in range(len(self.stack)):
            num = self.stack[i]
            if num>=max_num:
                max_num = num
                max_index = i

        return self.stack.pop(max_index)

# 0719  Hard  719. Find K th Smallest Pair Distance

In [ ]:
class Solution(object):
    def countPairsLTE(self, array, value):
        return sum(bisect.bisect_right(array, array[i] + value, lo = i) - i - 1 for i in range(len(array)))
        
    def smallestDistancePair(self, nums, k):
        nums.sort()
        low, high = min([nums[i + 1] - nums[i] for i in range(len(nums) - 1)]), nums[-1] - nums[0]
        while low < high:
            mid = (low + high) // 2
            if self.countPairsLTE(nums, mid) < k:
                low = mid + 1
            else:
                high = mid
        return low

# 0726  Hard  726. Number of Atoms

In [ ]:
class Solution:
    def countOfAtoms(self, formula):
        dic, coeff, stack, elem, cnt, i = collections.defaultdict(int), 1, [], "", 0, 0  
        for c in formula[::-1]:
            if c.isdigit():
                cnt += int(c) * (10 ** i)
                i += 1
            elif c == ")":
                stack.append(cnt)
                coeff *= cnt
                i = cnt = 0
            elif c == "(":
                coeff //= stack.pop()
                i = cnt = 0
            elif c.isupper():
                elem += c
                dic[elem[::-1]] += (cnt or 1) * coeff
                elem = ""
                i = cnt = 0
            elif c.islower():
                elem += c
        return "".join(k + str(v > 1 and v or "") for k, v in sorted(dic.items()))

# 0726 Hard 726 Number of Atoms

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections
import re


class Solution(object):
    def countOfAtoms(self, formula):
        """
        :type formula: str
        :rtype: str
        """
        parse = re.findall(r"([A-Z][a-z]*)(\d*)|(\()|(\))(\d*)", formula)
        stk = [collections.Counter()]
        for name, m1, left_open, right_open, m2 in parse:
            if name:
              stk[-1][name] += int(m1 or 1)
            if left_open:
              stk.append(collections.Counter())
            if right_open:
                top = stk.pop()
                for k, v in top.iteritems():
                  stk[-1][k] += v * int(m2 or 1)

        return "".join(name + (str(stk[-1][name]) if stk[-1][name] > 1 else '') \
                       for name in sorted(stk[-1]))

# 0727  Hard  727. Minimum Window Subsequence

In [ ]:
class Solution:
    def minWindow(self, S: str, T: str) -> str:
        def dfs(i, j):
            if j == len(T): return i
            if (i, j) not in memo:
                ind = S.find(T[j], i + 1)
                memo[(i, j)] = float('inf') if ind == -1 else dfs(ind, j + 1)
            return memo[(i, j)]
            
        l, res, memo = float('inf'), '', {}
        for i, s in enumerate(S):
            if s == T[0]:
                j = dfs(i, 1)
                if j - i < l:
                    l, res = j - i, S[i:j + 1]
        return res

# 0727 Hard 727 Minimum Window Subsequence

In [ ]:
# Time:  O(s * t)
# Space: O(s)

class Solution(object):
    def minWindow(self, S, T):
        """
        :type S: str
        :type T: str
        :rtype: str
        """
        lookup = [[None for _ in xrange(26)] for _ in xrange(len(S)+1)]
        find_char_next_pos = [None]*26
        for i in reversed(xrange(len(S))):
            find_char_next_pos[ord(S[i])-ord('a')] = i+1
            lookup[i] = list(find_char_next_pos)

        min_i, min_len = None, float("inf")
        for i in xrange(len(S)):
            if S[i] != T[0]:
                continue
            start = i
            for c in T:
                start = lookup[start][ord(c)-ord('a')]
                if start == None:
                    break
            else:
                if start-i < min_len:
                    min_i, min_len = i, start-i
        return S[min_i:min_i+min_len] if min_i is not None else ""

    
# Time:  O(s * t)
# Space: O(s)
class Solution2(object):
    def minWindow(self, S, T):
        """
        :type S: str
        :type T: str
        :rtype: str
        """
        dp = [[None for _ in xrange(len(S))] for _ in xrange(2)]
        for j, c in enumerate(S):
            if c == T[0]:
                dp[0][j] = j

        for i in xrange(1, len(T)):
            prev = None
            dp[i%2] = [None] * len(S)
            for j, c in enumerate(S):
                if prev is not None and c == T[i]:
                    dp[i%2][j] = prev
                if dp[(i-1)%2][j] is not None:
                    prev = dp[(i-1)%2][j]

        start, end = 0, len(S)
        for j, i in enumerate(dp[(len(T)-1)%2]):
            if i >= 0 and j-i < end-start:
                start, end = i, j
        return S[start:end+1] if end < len(S) else ""

# 0730  Hard  730. Count Different Palindromic Subsequences

In [ ]:
class Solution:
    def countPalindromicSubsequences(self, S):
        mod, memo = 10 ** 9 + 7, {}
        def dfs(i, j):
            if (i, j) not in memo:
                cnt = 0
                for x in "abcd":
                    try: l, r = S[i:j + 1].index(x) + i, S[i:j + 1].rindex(x) + i
                    except: continue  
                    cnt += l != r and dfs(l + 1, r - 1) + 2 or 1
                memo[(i, j)] = cnt % mod
            return memo[(i, j)]
        return dfs(0, len(S) - 1)

# 0730 Hard 730 Count Different Palindromic Subsequences

In [ ]:
# Time:  O(n^2)
# Space: O(n^2)

class Solution(object):
    def countPalindromicSubsequences(self, S):
        """
        :type S: str
        :rtype: int
        """
        def dp(i, j, prv, nxt, lookup):
            if lookup[i][j] is not None:
                return lookup[i][j]
            result = 1
            if i <= j:
                for x in xrange(4):
                    i0 = nxt[i][x]
                    j0 = prv[j][x]
                    if i <= i0 <= j:
                        result = (result + 1) % P
                    if None < i0 < j0:
                        result = (result + dp(i0+1, j0-1, prv, nxt, lookup)) % P
            result %= P
            lookup[i][j] = result
            return result

        prv = [None] * len(S)
        nxt = [None] * len(S)

        last = [None] * 4
        for i in xrange(len(S)):
            last[ord(S[i])-ord('a')] = i
            prv[i] = tuple(last)

        last = [None] * 4
        for i in reversed(xrange(len(S))):
            last[ord(S[i])-ord('a')] = i
            nxt[i] = tuple(last)

        P = 10**9 + 7
        lookup = [[None] * len(S) for _ in xrange(len(S))]
        return dp(0, len(S)-1, prv, nxt, lookup) - 1

# 0732  Hard  732. My Calendar III

In [ ]:
class MyCalendarThree:

    def __init__(self):
        self.times = []

    def book(self, start, end):
        bisect.insort(self.times, (start, 1))
        bisect.insort(self.times, (end, -1))
        res = cur = 0
        for _, x in self.times:
            cur += x
            res = max(res, cur)
        return res

# 0732 Hard 732. My Calendar III

In [ ]:
from sortedcontainers import SortedDict


class MyCalendarThree:
  def __init__(self):
    self.timeline = SortedDict()

  def book(self, start: int, end: int) -> int:
    self.timeline[start] = self.timeline.get(start, 0) + 1
    self.timeline[end] = self.timeline.get(end, 0) - 1

    ans = 0
    activeEvents = 0

    for count in self.timeline.values():
      activeEvents += count
      ans = max(ans, activeEvents)

# 0732 Hard 732 My Calendar III

In [ ]:
# Time:  O(nlogn) ~ O(n^2)
# Space: O(n)

import bisect


class MyCalendarThree(object):

    def __init__(self):
        self.__books = [[-1, 0]]
        self.__count = 0

    def book(self, start, end):
        """
        :type start: int
        :type end: int
        :rtype: int
        """
        i = bisect.bisect_right(self.__books, [start, float("inf")])
        if self.__books[i-1][0] == start:
            i -= 1
        else:
            self.__books.insert(i, [start, self.__books[i-1][1]])
        j = bisect.bisect_right(self.__books, [end, float("inf")])
        if self.__books[j-1][0] == end:
            j -= 1
        else:
            self.__books.insert(j, [end, self.__books[j-1][1]])            
        for k in xrange(i, j):
            self.__books[k][1] += 1
            self.__count = max(self.__count, self.__books[k][1])
        return self.__count


# Time:  O(n^2)
# Space: O(n)
class MyCalendarThree2(object):

    def __init__(self):
        self.__books = []


    def book(self, start, end):
        """
        :type start: int
        :type end: int
        :rtype: int
        """
        i = bisect.bisect_left(self.__books, (start, 1))
        if i < len(self.__books) and self.__books[i][0] == start:
            self.__books[i] = (self.__books[i][0], self.__books[i][1]+1)
        else:
            self.__books.insert(i, (start, 1))

        j = bisect.bisect_left(self.__books, (end, 1))
        if j < len(self.__books) and self.__books[j][0] == end:
            self.__books[j] = (self.__books[j][0], self.__books[j][1]-1)
        else:
            self.__books.insert(j, (end, -1))

        result, cnt = 0, 0
        for book in self.__books:
            cnt += book[1]
            result = max(result, cnt)
        return result

# 0736  Hard  736. Parse Lisp Expression

In [ ]:
class Solution:
    def evaluate(self, expression):
        scopes, items = [{}], [["root"]]
        for item in expression.replace(")", " )").split():
            if item[0] == "(":
                items.append([item[1:]])
                if item[1:] == "let":
                    scopes.append(dict(scopes[-1]))
                continue
            elif item == ")": 
                if items[-1][0] == "add":
                    item = str(int(items[-1][1]) + int(items[-1][-1]))
                elif items[-1][0] == "mult":
                    item = str(int(items[-1][1]) * int(items[-1][-1]))
                else:
                    item = items[-1][-1]
                    if item in scopes[-1]:
                        item = scopes[-1][item]
                    scopes.pop()
                items.pop()
            if item in scopes[-1] and (items[-1][0] != "let" or len(items[-1]) % 2 == 0):
                item = scopes[-1][item]
            if items[-1][0] == "let" and item.lstrip("-").isdigit():
                scopes[-1][items[-1][-1]] = item
            items[-1].append(item)
        return int(items[-1][-1])